## Packages

In [ ]:
!pip install keybert
!python -m spacy download pt
!pip install sentence_transformers
!pip install pysinonimos
!sudo apt-get install libomp-dev
!pip install spacy-wordnet
!pip install rank-bm25
!pip install hnswlib

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


## Libs

In [ ]:
import spacy
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import os
from rank_bm25 import BM25Okapi
import pandas as pd
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer, util
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
from zipfile import ZipFile
import logging
import operator
import csv
import re
import sys
import torch
from operator import itemgetter
import math
import gzip
import spacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import nltk
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import time
import itertools
import pickle
nltk.download('stopwords')
nltk.download('punkt')
stop_words=set(stopwords.words("portuguese"))
nlp = spacy.load('pt')
nltk.download('wordnet')
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
nltk.download('omw')
nlp = spacy.load('pt')
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
os.chdir('/content/drive/MyDrive/Data_PHD')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Package omw is already up-to-date!


# Load models 

In [ ]:
def keyextractor (doc):
  keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=stop_words)
  first_tuple_elements = []
  for i in keywords:
    first_tuple_elements.append(i[0])
  return first_tuple_elements

def lemmatizer (x): 
  lista = []
  nlp = spacy.load("pt")
  for token in x:
    for ff in nlp(token):
      lemma = ff.lemma_
      lista.append(lemma)
  return lista

bi_encoder = SentenceTransformer('/content/drive/MyDrive/output/bi-encoder/qqp_cross_domain_neuralmind-bert-base-portuguese-cased-2021-01-26_10-26-24')
#bi_encoder = SentenceTransformer('bert-base-multilingual-cased')
cross_encoder = CrossEncoder('/content/drive/MyDrive/output/cross-encoder/stsb_indomain_neuralmind-bert-base-portuguese-cased-2021-01-25_19-42-07')
model = SentenceTransformer('/content/drive/MyDrive/output/bi-encoder/qqp_cross_domain_neuralmind-bert-base-portuguese-cased-2021-01-26_10-26-24') 
corpus_embeddings = pickle.load( open( "embeddings.pkl", "rb" ) )
dicionario = pickle.load( open( "dicionario.pkl", "rb" ) )
lemmas = pickle.load( open( "lemmas.pkl", "rb" ) )
kw_model_base = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')
    

## test set and corpus

In [ ]:


passages = '/content/drive/MyDrive/Data_PHD/dados_new.csv'
df = pd.read_csv(passages)
df_full = df['sentences1'].tolist()
x = pd.read_csv('teste_bert - question.csv')
data = x['question'].tolist()
#corpus_embeddings = model.encode(df_full, show_progress_bar=True,convert_to_numpy=True)

# HoPE Model

In [ ]:
 corpus_embeddings = model.encode(df_full, show_progress_bar=True, convert_to_numpy=True)

In [ ]:
import hnswlib
embedding_size = 768
index_path = "./hnswlib.index"
index = hnswlib.Index(space = 'cosine', dim = embedding_size)


In [ ]:
if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
else:
    ### Create the HNSWLIB index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements = len(corpus_embeddings), ef_construction = 500, M = 150)

    # Then we train the index to find a suitable clustering
    index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))

    print("Saving index to:", index_path)
    index.save_index(index_path)
    index.set_ef(220) 
max_corpus_size = 4000

Loading index...


In [ ]:
frases = [[i] for i in data]

In [ ]:
top_k_hits = 8
for documentos in frases:
  for i in documentos:
    top_k = 10
    start_time = time.time()
    question_embedding = bi_encoder.encode(i)
    corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)
    hits = [{'corpus_id': id, 'score': 1-score} for id, score in zip(corpus_ids[0], distances[0])]
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    #hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  #   hits = hits[0] 
    cross_inp = [[i, df_full[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
      hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    lista = {}
    lista_ = []
    for hit in hits[0:10]:
      lista[df_full[hit['corpus_id']]] = (10*hit['score'])
    print('input', i) 
    print(lista)
    enriched_sentence = {}
    for token in documentos:
      sentence = nlp(token)
      for token in sentence:
        synsets = token._.wordnet.synsets()
        if not synsets:
          enriched_sentence[token] = token.text
        else:
          lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names('por')][0:4]
          new_doc = []
          for w in lemmas_for_synset:
             if w in dicionario:
               new_doc.append(w)
          enriched_sentence[token] = list(set(new_doc))
      dictlist =[]
      for key, value in enriched_sentence.items():
        temp = [key,value]
        dictlist.append(temp)
        flat_list = [item for sublist in dictlist for item in sublist]
        r1 = re.findall(r"[a-zA-Z\u00C0-\u00FF]*",str(flat_list))
        r2 = ' '.join(r1)
    query =(r2.split())
    tokenized_corpus = [doc.split(" ") for doc in lista.keys()]
    bm25 = BM25Okapi(tokenized_corpus)
    doc_scores = bm25.get_scores(query)
    doc_scores.sort()
    doc_scores = doc_scores[::-1]
    res = bm25.get_top_n(query, tokenized_corpus, n=10)
    lista_3 = []
    for x in res: 
      lista_2 = TreebankWordDetokenizer().detokenize(x)
      lista_3.append(lista_2)
  zip_iterator = zip( lista_3, doc_scores)
  a_dictionary = dict(zip_iterator)
  resultado_f = {k: lista.get(k, 0) + a_dictionary.get(k, 0) for k in set(lista) | set(a_dictionary)} 
  print(max(resultado_f.items(), key=operator.itemgetter(1))[0])
  #print(resultado_f)

  print('---------------------------------------------------')


input Posso beber bebida alcólica?
{'Não consumir álcool .': 8.124842047691345, 'Em decorrência do risco da síndrome alcoólica fetal, o consumo de álcool durante a gravidez \ndeve ser proibido, mesmo que seja ocasional.': 6.966831088066101, 'Na prática, bebidas do grupo efervescente tem baixa \nconcentração alcoólica, enquanto os destilados têm alto teor alcoólico.': 6.414738893508911, 'O álcool pode levar a síndrome alcoólica fetal e deficiência mental."A dose limítrofe é \nde 28,35 g/álcool absoluto/dia, o que corresponde a 1 drinque/dia."': 6.446929574012756, 'Dessa forma, sua utilização durante a gestação deve ser contraindicada, uma vez que não foi \nencontrada nenhuma evidência de que existe uma quantidade segura de álcool que possa \nser consumida durante a gravidez.': 5.98455011844635, 'Tabaco, drogas, álcool e radiografias Fumar, consumir drogas (lícitas e/ou ilícitas) \nou álcool durante a gravidez nunca é aconselhável.': 6.41068160533905, 'No entanto, consumo eventual e mode

## Output

In [ ]:
input Posso beber bebida alcólica? 
Não consumir álcool .
---------------------------------------------------
input Quais alimentos devo comer?
 Os alimentos oferecidos às crianças devem ser preparados pouco 
antes do consumo. nunca oferecer restos de uma refeição.
---------------------------------------------------
input O que é um plano de parto?
Na realidade, a opção de iniciar precocemente o pré-natal é baseada nas possibilidades de acesso ao sistema de saúde.
---------------------------------------------------
input Frutas fazem bem?
No Sudeste, frutas à tarde e feijão à 
noite fazem mal.
---------------------------------------------------
input Qual sua opinião sobre bacon?
Evitar bacon,frios e
todos os dias.
---------------------------------------------------
input Qual a relação de leite e derivados
Existe uma relação direta entre duração da amamentação exclusiva 
e escolaridade materna.
---------------------------------------------------
input É possivel ter anemia?
Por esse motivo, a não ser que haja suplementação exógena adequada, a maioria das gestantes evolui para anemia ferropriva.
---------------------------------------------------
input o exercício físico pode atrapalhar
Atividade física e profissional Exercícios muito intensos podem dificultar a concepção e levar a crescimento fetal restrito.
---------------------------------------------------
input Por quanto tempo fzer exercicios?
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Como é uma alimentação saudavel?
A alimentação deve ser saudável e balanceada, estimulando a ingestão de água e fibras.
---------------------------------------------------
input Qual sua opinião sobre cereais?
A melhor opção é perguntar sobre a percepção da paciente a respeito de sua saúde atual, sua certeza de gravidez, e buscar eventuais queixas.
---------------------------------------------------
input Arroz e massas fazem bem?
A mistura de arroz com feijão é e massas.
---------------------------------------------------
input Qual o tempo ideal para começar a pratica de exercicios na gestação?
Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.
---------------------------------------------------
input Quando devo interromper os exercicios?
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Quais sao os treinos na gestação?
Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.
---------------------------------------------------
input Sobre exercicios com a vagina?
Como fazer os exercícios perineais
Contraia a vagina e o ânus, como se estivesse puxandoos para dentro e para cima do seu corpo.
---------------------------------------------------
input Até que idade posso ter filho?
A prevalência do aleitamento materno é baixa, sua duração é curta e o 
aleitamento materno exclusivo até o sexto mês de vida é raro.
---------------------------------------------------
input Como posso confirmar se estou gravida
Sim, desde que você esteja realizando o pré-natal e não apresente nenhuma
complicação na gravidez.
---------------------------------------------------
input Posso perder o bebe?
A mãe que tem 
o vírus não deve amamentar o bebê, porque há risco de transmissão da mãe para o filho.
---------------------------------------------------
input E sobre intervençoes?
Entre o nascimento e a última procedência, pode haver uma gama de informações relevantes, principalmente no que se refere às infecções e infestações.Ainda nesse item deve-se questionar o eventual tipo de assistência pré-natal recebida até o momento e as intervenções que já foram iniciadas.
---------------------------------------------------
input Atendimentos mensais  no pre natal?
Cuidado Recomendado
pré-natal Atendimentos mensais até a 28ª semana
Atendimentos quinzenais da 28ª até a 36ª semana
Atendimentos semanais da 36ª semana até o parto e nascimento
Puerpério Primeira consulta até o 7º dia pósparto
Segunda consulta entre o 30º e 40º dias pósparto Os atendimentos subsequentes devem seguir a um roteiro similar ao da primeira consulta, porém mais simplificado com sempre maior atenção para os múltiplos fatores que podem determinar o desenvolvimento da gravidez.
---------------------------------------------------
input E sobre aplicativos para idade gestacional? eles funcionam?
Você poderá realizar todos os exercícios orientados paras gestantes.
---------------------------------------------------
input E sobre vacinas, quais tenho q fazer?
A exceção está nas gestantes com problemas imunológicos, que devem receber medicações específicas 
até o 7o mês (heparina, aspirina, corticoides etc.).
---------------------------------------------------
input Tenho que fazer a vacina da febre amarela?
Vacinação 
A mulher deve tomar vacina contra sarampo, caxumba e rubéola (MMR), caso não seja 
imune a essas doenças, e deve ser orientada a não engravidar nos três meses seguintes.
---------------------------------------------------
input Tenho HIV, quais os problemas disso?
Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.
---------------------------------------------------
input Quais são os exames que preciso fazer?
Na avaliação inicial da grávida, deverão ser discutidos seu estado geral e os exames 
laboratoriais a serem solicitados.
---------------------------------------------------
input De quanto em quanto tempo devo ir consulta
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input O que garante o pre-natal?
Com intuito de atrair as gestantes o mais precocemente possível, o Ministério da Saúde tem estimulado as instituições com benefícios extras quando a grávida inicia 
rapidamente o pré-natal e garantese o mínimo de consultas, exames, vacinação e retorno no puerpério.
---------------------------------------------------
input Gostaria de saber sobre sindromes hipertensivas?
Doenças como diabetes, hiper ou hipotireoidismo podem ser suspeitadas ou informadas no questionário.
---------------------------------------------------
input Como devo tratar a sifilis?
As principais viroses a serem pesquisadas são rubéola, doença de inclusão citomegálica, 
herpes simples, hepatites B e C, papilomavírus humano, influenza e HIV.
---------------------------------------------------
input Quais os melhores exercicios aerobicos?
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Quanto tempo deve durar os exercicios?
A duração mediana do Aleitamento Materno E foi de 
54 dias (0,7 a 89 dias) e a do Aleitamento Materno de 342 dias (293 a 601 dias).
---------------------------------------------------
input E bom beber liquidos ?
Beba líquidos em abundância, principalmente leite, suco de frutas e água,
pois a amamentação dá muita sede.
---------------------------------------------------
input Posso comer castanhas e nozes?
 “comida de animais” – Norte folhas verdes em geral. 
Nordeste batata doce, folhas verdes em geral, jerimum/abóbora.
---------------------------------------------------
input o que e bom evitar?
Orientações sobre evitar o decúbito horizontal logo após as refeições, assim como fracionar 
a dieta, evitando a ingestão de alimentos como café, chá, chocolate e frituras, normalmente 
são suficientes.
---------------------------------------------------
input E importante relaxar?
Prático de Exercícios
Relaxamento
É muito importante que após os exercícios você faça um relaxamento para se
recuperar e aumentarsuainteração com o bebê.
---------------------------------------------------
input Tenho que realizar uma dieta na gestação?
Nutrição da gestante
A mulher que pretende engravidar tem necessidades diferenciadas na preconcepção e 
conforme o trimestre gestacional.
---------------------------------------------------
input Gravidez
A mulher vacinada deve aguardar quatro semanas para iniciar nova gestação.
---------------------------------------------------
input Sobre omega 3?
2 – A amamentação exclusiva ainda é pouco praticada em todas 
as regiões.
---------------------------------------------------
input tempo de exercicios?
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input A sobrecarga emocional e fisica esta grande
A gravidez é um período de sobrecarga física e emocional, e corresponde a um trabalho braçal de 
intensidade média.
---------------------------------------------------
input Como saber se tenho doenças?
Caso essa situação não seja possível, a gestante deve ser rastreada quanto a sua imunidade 
contra a rubéola no início do pré-natal.
---------------------------------------------------
input Posso fazer cirurgia bariatrica
As cirurgias bariátricas podem ser máabsortivas e restritivas, como ocorre com aquela que utiliza banda gástrica por videolaparoscopia e que, durante o pré-natal, pode exigir ajustes.
---------------------------------------------------
input Qual a importnacia do pre-natal?
A falta de mudança dos BCF nas situações descritas pode ocorrer por se tratar do momento 
de sono fetal.
---------------------------------------------------
input Qual a melhor idade para a muher ter um filho?
Após 6 meses a 1 ano do nascimento do bebê, o esperado é que você retorne
ao peso quetinha antes de engravidar.
---------------------------------------------------
input O q nao e aconselhavel na gravidez?
O uso do ácido fólico na préconcepção costuma ser recomendado para qualquer mulher que pretenda engravidar, como já foi exposto .
---------------------------------------------------
input Comer carne crua faz mal?
 “proibidos pela religião” – Norte camarão, carne de porco.
---------------------------------------------------
input o sobrepeso pode causar problemas?
No entanto, cerca de 2/3 das mulheres ganham mais peso do que o recomendado, 
o que leva a complicações durante a gestação, além de contribuir para a retenção de peso pós
parto e, com isso, para o desenvolvimento da obesidade e suas complicações ao longo da vida.
---------------------------------------------------
input sou obesa, posso ter filho?
Gestantes com sobrepeso devem 
ganhar até 0,9 kg no primeiro trimestre, e gestantes obesas não necessitam ganhar peso no 
primeiro trimestre.
---------------------------------------------------
input E sobre a vacina contra o tetano?
Febre tifoide Não existem dados sobre o uso da vacina anti tifoide em gestantes.
---------------------------------------------------
input Quais os exames que eu devo fazer?
O exame deve ser realizado por volta de 35 semanas, 
para que se promova a profilaxia intraparto.
---------------------------------------------------
input Qual a importancia da do diagnostivo na gravidez?
A amamentação exclusiva é importante também na diminuição da 
fertilidade após o parto.
---------------------------------------------------
input Qual a pressao arterial ideal?
É importante frisar que uma medida de pressão arterial de 130 x 80 mmHg é aceitável no 
final da gravidez, mas deve chamar atenção no segundo trimestre, quando são observados 
valores mais baixos (ao redor de 105 x 60 mmHg).
---------------------------------------------------
input Problemas cardiacos?
Neste ponto é necessário ressaltar que rastrear processos patológicos latentes, entre eles a cardiopatia, as endocrinopatias e erros posturais, 
é de extrema importância.
---------------------------------------------------
input o que é sopro?
CORIONICIDADE
É muito importante, quando se fala em corionicidade, saber a zigoticidade.
---------------------------------------------------
input Queria saber mais sobre ovarios
Nos casos de gestação repetida, questionar também sobre a evolução do puerpério, 
tentando identificar informações sobre depressão puerperal, infecções e a experiência com 
a amamentação.
---------------------------------------------------
input A violencia pode prejudicar minha gravidez
Muitos medicamentos e fitoterápicos podem ter efeitos danosos para o feto em qualquer momento da gravidez, causando malformações congênitas (teratogênese), prejuízo para o crescimento e o desenvolvimento funcional do feto, ou efeitos tóxicos sobre os tecidos fetais.
---------------------------------------------------
input Tive problemas ginecologicos, posso engravidar?
À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.
---------------------------------------------------
input Tive problemas  obstetricos, posso engravidar?
À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.
---------------------------------------------------
input Problemas familiares podem me prejudicar?
Intercorrências clínicocirúrgicas
A ocorrência de algumas doenças durante a gestação pode interferir no prognóstico materno
fetal.
---------------------------------------------------
input As mamas se modificam na gravidez?
 Adrenais
As adrenais não sofrem mudanças morfológicas durante a gestação.
---------------------------------------------------
input Exames vaginais 
Complementase o exame físico obstétrico com o toque vaginal (na maioria das vezes, 
bidigital), combinado com a palpação uterina.
---------------------------------------------------
input Posso usar filtro solar?
O filtro solar deve ser reaplicado pelo menos três vezes ao dia, 
mesmo sem exposição ao sol.
---------------------------------------------------
input Varizes na gravidez
O evoluir da gestação mimetiza manifestações clínicas de doenças, na 
decorrência das adaptações gravídicas do organismo materno.
---------------------------------------------------
input Tenho que preparar as mamas para amamentar?
Depois que a gestante compreender a importância da amamentação, ficará mais fácil aceitar as medidas de preparo das mamas para o aleitamento, evitando que ocorram fissuras mamilares e haja descontinuidade da amamentação.
---------------------------------------------------
input Viagens estao permitidas?
Não há restrição 
de viagens aéreas até a 34a semana de gravidez.
---------------------------------------------------
input Atividades fisicas sao importantes?
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Posso fazer sexo na gravidez?
A revolução sexual da década de 1960 questionou a sexualidade da mulher 
resultando na liberdade de iniciar a atividade sexual sem vínculo obrigatório de casamento.
---------------------------------------------------
input Exames rotineiros sao importantens na gravidez?
Exame físico ginecológico deve-se seguir o padrão habitual de todo exame 
ginecológico.
---------------------------------------------------
input o que e o cartao prenatal?
Administrativamente, as consultas prénatais se dividem em primeira consulta (também 
chamada de caso novo de pré-natal) e os retornos.
---------------------------------------------------
input Exercicio fisico 
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Exercicio fisico especial
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Exame gineco obsetrico
Exame físico ginecoobstétrico
Em termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal 
deverá considerar o exame das mamas e o exame obstétrico propriamente dito.
---------------------------------------------------
input Quais condutas e orientações na gestação?
O profissional de saúde deve avaliar se os mamilos estão invertidos ou não, o que vai ditar as orientações e informações 
que ele precisa fornecer à gestante quanto aos cuidados com as mamas e a amamentação.
---------------------------------------------------
input Ecografia
Ecocardiografia fetal
A ecocardiografia fetal é um exame anatômico e funcional do coração fetal."Pode ser 
realizado a partir de 18 semanas de gestação, mas o período ideal é o da 28a semana."
---------------------------------------------------

In [ ]:
## Clustering sentences

input Posso beber bebida alcólica?
{'Não consumir álcool .': 0.8124842047691345, 'Em decorrência do risco da síndrome alcoólica fetal, o consumo de álcool durante a gravidez \ndeve ser proibido, mesmo que seja ocasional.': 0.6966831088066101, 'Na prática, bebidas do grupo efervescente tem baixa \nconcentração alcoólica, enquanto os destilados têm alto teor alcoólico.': 0.6414738893508911, 'O álcool pode levar a síndrome alcoólica fetal e deficiência mental."A dose limítrofe é \nde 28,35 g/álcool absoluto/dia, o que corresponde a 1 drinque/dia."': 0.6446929574012756, 'Dessa forma, sua utilização durante a gestação deve ser contraindicada, uma vez que não foi \nencontrada nenhuma evidência de que existe uma quantidade segura de álcool que possa \nser consumida durante a gravidez.': 0.598455011844635, 'Tabaco, drogas, álcool e radiografias Fumar, consumir drogas (lícitas e/ou ilícitas) \nou álcool durante a gravidez nunca é aconselhável.': 0.641068160533905, 'No entanto, consumo eventual e moderado de álcool (0,5g de álcool por quilo de peso da mãe por dia, o que corresponde a aproximadamente uma taça de \nvinho ou duas latas de cerveja por dia) é considerado compatível com a amamentação.': 0.6457024812698364, 'Não existe \nindicação de restringir a ingestão de alimentos pelo fato da criança encontrar\nse doente.': 0.596886932849884, 'Hábitos\ndeve-se perguntar à paciente sobre tabagismo, etilismo e uso de drogas ilícitas.': 0.5846437811851501, '6.3.4 Uso de medicamentos, drogas de abuso, cigarro e álcool\nA utilização, pela mãe, da maioria dos medicamentos é compatível com a amamentação. \npoucos são os formalmente contraindicados e alguns requerem cautela ao serem utiliza\ndos durante a amamentação, devido aos riscos de efeitos adversos nos lactentes e/ou na \nlactação.': 0.5630900263786316}
Não consumir álcool .
---------------------------------------------------
input Quais alimentos devo comer?
{' Os alimentos oferecidos às crianças devem ser preparados pouco \nantes do consumo. nunca oferecer restos de uma refeição.': 0.48340439796447754, 'O \nconsumo de outros alimentos varia de acordo com o local, mas quase \nsempre inclui biscoito, arroz, feijão, macarrão e batata.': 0.4432147741317749, 'As pacientes que fazem três refeições ao dia (sem pular ou subtrair nenhuma delas), consistindo \nde frutas, vegetais, grãos integrais, produtos pobres em gorduras e algumas fontes de \nproteína, devem estar bem nutridas, ao contrário das que se alimentam em fast foods, pulam \nvárias refeições por semana e consomem salgadinhos, bolos e doces em várias ocasiões.': 0.47721588611602783, ' Esses alimentos não são bons para a nutrição da criança e \ncompetem com alimentos mais nutritivos.': 0.42400825023651123, 'Os novos alimentos oferecidos à \ncriança não devem ser muito doces ou salgados.': 0.4210329055786133, 'PASSO 7 – Estimular o consumo diário de frutas, verduras e legumes nas \nrefeições.': 0.46616172790527344, 'Orienta-se dieta fracionada com menor volume de alimento em alguns casos, sugerindo a \ningestão de substâncias pastosas.': 0.43233776092529297, '5 – As dietas são, em geral, consideradas adequadas quanto ao \nconteúdo energético.': 0.45824503898620605, 'Nesse período, é recomendável uma dieta com mais \ncalorias e proteínas.': 0.4693549871444702}
 Os alimentos oferecidos às crianças devem ser preparados pouco 
antes do consumo. nunca oferecer restos de uma refeição.
---------------------------------------------------
input O que é um plano de parto?
{'A assistência pré-natal é um conjunto de medidas de natureza médica, social, psicológica \ne de cuidados gerais que visa propiciar à mulher gestante o desenvolvimento saudável \nda gravidez.': 0.4219992160797119, 'Respeitando-se o conceito de bemestar físico, psíquico e social, do ponto de vista obstétrico, a gestação deveria ser programada com uma avaliação précocepcional, visando ao momento mais adequado para a sua procura.': 0.45388704538345337, 'Administrativamente, as consultas prénatais se dividem em primeira consulta (também \nchamada de caso novo de pré-natal) e os retornos.': 0.43920469284057617, 'Na realidade, a opção de iniciar precocemente o pré-natal é baseada nas possibilidades de acesso ao sistema de saúde.': 0.5206132531166077, 'Com intuito de atrair as gestantes o mais precocemente possível, o Ministério da Saúde tem estimulado as instituições com benefícios extras quando a grávida inicia \nrapidamente o pré-natal e garantese o mínimo de consultas, exames, vacinação e retorno no puerpério.': 0.44785642623901367, 'Para as gestantes de alto risco, a participação deve ser coerente com o plano de cuidados definido pela equipe especializada.': 0.46692341566085815, 'Justificando plenamente a expressão “Gestação de 12 \nmeses – One year pregnancy” (Peixoto, 2009), a assistência preconcepcional compõese de \ncuidados dirigidos ao casal para que se possa criar um ambiente favorável para a gravidez \nfutura.': 0.4672853350639343, 'Uma ação que demonstra a qualidade do fluxo na RAS é a vinculação da gestante à maternidade de referência, dependendo da estratificação de risco.': 0.4256308674812317, 'Estímulo da participação do parceiro ou dos familiares no pré-natal\ndeve-se dar ao parceiro da gestante a opção de participar do pré-natal."São inúmeras as \nvantagens dessa participação para a gestante, para o parceiro e para o futuro filho."': 0.4818834066390991, 'Algumas delas, em fases iniciais, permitem o tratamento intraútero, que proporciona ao recém-nascido \numa vida normal.': 0.45144587755203247}
Na realidade, a opção de iniciar precocemente o pré-natal é baseada nas possibilidades de acesso ao sistema de saúde.
---------------------------------------------------
input Frutas fazem bem?
{'No Sudeste, frutas à tarde e feijão à \nnoite fazem mal.': 0.5585911870002747, 'A gestante deve ingerir frutas, verduras e legumes, leite e derivados, o qual contêm nutrientes \nimportantes para o crescimento fetal, da mesma forma que podem contribuir para evitar o \nganho excessivo de peso.': 0.4709351658821106, 'O consumo de \nfrutas, verduras e folhosos verdes variados não constitui um hábito entre as \ncrianças no primeiro ano de vida.': 0.48530519008636475, 'Na Região CentroOeste, são considerados bons os seguintes \nalimentos hortaliças (destacandose a cenoura e a beterraba), frutas, carne, \nleite e derivados.': 0.5098977088928223, 'Além da suplementação, \nlembramos que o ácido fólico está presente nas folhas de cor verdeescura, entre elas o \nespinafre, a escarola, a couve, os brócolis, além de leguminosas, castanhas e frutas cítricas.': 0.49496960639953613, ' Esses alimentos não são bons para a nutrição da criança e \ncompetem com alimentos mais nutritivos.': 0.5015076994895935, 'Na Região Sul, observase que os alimentos considerados bons são \naqueles que sustentam, as comidas de sal.': 0.5384491682052612, 'Há também referência a \nalimentos frescos como alimentos bons.': 0.5579826235771179, 'Anemia  Comerdiariamente alimentos fontes de Ferro, como carne por vermelha, fígado, folhas verdes escuras e leguminosas\ncarência (feijão, soja, lentilha). "Consumir junto com os alimentos fontes de ferro, suco ou\nprevenir fruta cítrica (laranja, limão, acerola)."': 0.5404980182647705, 'Por outro lado, \nalimentos oferecidos como recompensa são os prediletos (Birch et al., \n1980).': 0.4747557044029236}
No Sudeste, frutas à tarde e feijão à 
noite fazem mal.
---------------------------------------------------
input Qual sua opinião sobre bacon?
{'Evitar bacon,frios e\ntodos os dias.': 0.5291835069656372, 'Na Região Sul, observase que os alimentos considerados bons são \naqueles que sustentam, as comidas de sal.': 0.4986172914505005, 'No \nNordeste, embora as mães tenham essa mesma opinião, elas não reconhecem \nesses alimentos como fontes de vitamina A. Os dados de todas as regiões \nsugerem que esses alimentos não são prioridades de compra e nem de \nconsumo.': 0.46497809886932373, 'Infelizmente, muitos desses alimentos não são os mais \nrecomendados pelo excesso de açúcar, gordura ou sal.': 0.46196448802948, 'Uma vez acostumados com \num certo nível de açúcar ou sal, a tendência da criança é de rejeitar outras \nformas de preparação do alimento (Sullivan e Birch, 1990).': 0.46183741092681885, 'Ainda não se sabe se experiências intrauterinas podem afetar futuras \npreferências de sabores.': 0.4673287272453308, ' deve-se evitar dar à criança alimentos muito condimentados \n(pimenta, mostarda, “catchup”, temperos industrializados).': 0.4829521179199219, 'Na Região Sul, a banana consumida após o almoço é tida \npor alguns como indigesta.': 0.4813796281814575, ' “proibidos pela religião” – Norte camarão, carne de porco.': 0.48102885484695435, 'Há também referência a \nalimentos frescos como alimentos bons.': 0.5114943385124207}
Evitar bacon,frios e
todos os dias.
---------------------------------------------------
input Qual a relação de leite e derivados
{'Na Região CentroOeste, verificouse que o alimento mais aceito \ndurante a doença é o leite (materno ou não humano) e a água de coco.': 0.5856810212135315, 'A \nalimentação das crianças nessa faixa de idade é basicamente láctea (leite \nmaterno e/ou de vaca), acrescido de açúcar e espessantes.': 0.5454527139663696, 'Sabe-se que muitas mães não consideram água, chás ou outros líquidos \ncomo complemento do leite materno, fazendo com que elas respondam que \ndão só o peito quando na realidade as crianças recebem também outros \nlíquidos (Monteiro et al., 1997).': 0.5303604006767273, ' Aleitamento materno – quando a criança recebe leite materno, \ndiretamente do seio ou extraído, independente de estar recebendo \nqualquer alimento ou líquido, incluindo leite nãohumano.': 0.576603353023529, '3.3 Alimentação Complementar \n \n Alimentos complementares são quaisquer alimentos que não o leite \nhumano oferecidos à criança amamentada.': 0.5912944674491882, '6.1 Definições\nO Ministério da Saúde adota as seguintes definições de Aleitamento Materno, preconizadas pela OMS e re\nconhecidas em todo o mundo2\nAleitamento MaternoE – quando a criança recebe somente leite materno, direto da mama ou ordenhado, ou leite humano de outra fonte, sem outros líquidos ou sólidos, com exceção de gotas ou xaropes contendo vitaminas, sais de reidratação oral, suplementos minerais ou medi\ncamentos. "\nAleitamento Materno predominante – quando a criança recebe, além do leite materno, água ou bebidas à base de água (água adocicada, chás, infusões), sucos de frutas e fluidos rituais."\n"\tAleitamento Materno – quando a criança recebe leite materno (direto da mama ou ordenhado), indepen\ndentemente de estar recebendo ou não outros alimentos."\n"\tAleitamento Materno complementado – quando a criança recebe, além do leite materno, alimentos com\nplementares, que são alimentos sólidos ou semissólidos que complementam o leite ma\nterno."\n"Nesta categoria a criança pode estar recebendo, além do leite materno, outro tipo de leite, mas este não é considerado alimento complementar."\n"\tAleitamento Materno misto ou parcial – quando a criança recebe leite materno e outros tipos de leite."': 0.530632734298706, 'Existe uma relação direta entre duração da amamentação exclusiva \ne escolaridade materna.': 0.6586726307868958, '6.3.2 Noções sobre o leite materno\nNos primeiros dias, o leite materno é chamado colostro, e contém mais proteínas e menos \ngorduras que o leite maduro.': 0.5357038378715515, '3.2 Aleitamento Materno 3.2.1 Definições Este documento adota as mesmas categorias de aleitamento materno sugeridas pela OMS, assim definidas (Organización Panamericana de la Salud/Organización Mundial de la Salud, 1991)  Aleitamento materno exclusivo – quando a criança recebe somente leite materno, diretamente da mama ou extraído, e nenhum outro líquido ou sólido, com exceção de gotas ou xaropes de vitaminas, minerais e/ou medicamentos.': 0.538141131401062, 'Para tanto, devem se levar em consideração tanto componentes maternos como do produto da \nconcepção (feto e anexos).': 0.5368564128875732}
Existe uma relação direta entre duração da amamentação exclusiva 
e escolaridade materna.
---------------------------------------------------
input É possivel ter anemia?
{'Alguns efeitos da anemia por \ndeficiência de ferro podem ser irreversíveis (Lozzof et al., 1987).': 0.6268466711044312, 'A isoimunização Rh pode conduzir à destruição dos glóbulos vermelhos do feto, levando a \numa anemia grave.': 0.5605878233909607, 'A anorexia na criança doente é um fenômeno universal e pode \npersistir além do episódio da doença.': 0.5761591792106628, 'Por esse motivo, a não ser que haja suplementação exógena adequada, a maioria das gestantes evolui para anemia ferropriva.': 0.6833062767982483, 'Conclusões\nA gestação após a cirurgia bariátrica é segura para a mãe e para o feto, embora possa levar \na desnutrição e anemia maternas, requerendo seguimento das deficiências de nutrientes na \npréconcepção, na gravidez e no puerpério.': 0.5585275292396545, 'Quando doenças concomitantes são diagnosticadas durante o pré-natal, tenta\nse compensar ou curar aquelas que possam ser controladas ou curadas com a gravidez em \nevolução.': 0.5523844957351685, 'Ainda que não existam estudos que comprovem sua associação com malformações congênitas, relatase infecção fetal pósvacinação materna, apesar de rara.Caso a gestante tenha sido exposta a risco elevado de contrair a doença, ela pode ser vacinada, já que as sequelas da infecção são mais importantes que o risco de comprometimento fetal.': 0.5680072903633118, 'Período de mineralização óssea momento em que ocorre a deposição de cálcio no  osso fetal e em que aumenta a necessidade de cálcio a ser suplementado até a lactação, \na fim de minimizar estados de hipocalcemia relacionados às demandas fetal e neonatal."Assim, as vantagens do pré-natal são identificar doenças que já estavam presentes \nno organismo, porém, evoluindo de forma silenciosa, como a hipertensão arterial, \ndiabetes, cardiopatias, anemias e doenças infecciosas, fazer seu diagnóstico \npermite a adoção de medidas de tratamento que evitam maior prejuízo à mulher, \nnão só durante a gestação, mas por toda sua vida. detectar alterações fetais, como \nmalformações e problemas de crescimento e de desenvolvimento."': 0.5639755129814148, 'Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.': 0.5864207744598389, 'Embora essas alterações sejam consideradas fisiológicas e normais, elas podem mascarar ou piorar doenças preexistentes.': 0.5815438628196716}
Por esse motivo, a não ser que haja suplementação exógena adequada, a maioria das gestantes evolui para anemia ferropriva.
---------------------------------------------------
input o exercício físico pode atrapalhar
{'Atividade física e profissional Exercícios muito intensos podem dificultar a concepção e levar a crescimento fetal restrito.': 0.6755142211914062, 'Devem ser evitados exercícios que representem algum tipo de risco, como a musculação por \nflexão de coluna e por impacto.': 0.5656006336212158, 'A prática de exercício físico leve ou moderado durante a gravidez é segura tanto para\na mãe quanto para o bebê.': 0.5511335730552673, 'A prática de exercício físico leve ou moderado durante a gravidez é segura tanto para \na mãe quanto para o bebê.': 0.5511335730552673, 'Evite exercícios que tenham risco de queda, bater a barriga ou muito contato físico, como por exemplo, esportes com bola, saltos, lutas marciais, ciclismo, equitação.': 0.5688101053237915, 'Evite exercícios que tenham risco de queda, bater a barriga ou muito contato físico, como\npor exemplo, esportes com bola, saltos, lutas marciais, ciclismo, equitação.': 0.5688101053237915, 'Evite exercícios que tenham risco de queda, bater a barriga ou muito contato físico, como \npor exemplo, esportes com bola, saltos, lutas marciais, ciclismo, equitação.': 0.5688101053237915, 'Da \nmesma forma, deve-se evitar a prática de esportes coletivos com risco de contato físico e \naqueles que envolvem atividade física extenuante.': 0.6108410358428955, 'Também são contraindicados os esportes \ncom risco de queda, como o hipismo e o esqui.': 0.571373462677002}
Atividade física e profissional Exercícios muito intensos podem dificultar a concepção e levar a crescimento fetal restrito.
---------------------------------------------------
input Por quanto tempo fzer exercicios?
{'Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6405569314956665, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.697513997554779, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5954394936561584, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5954394936561584, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.5583480000495911, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.5583480000495911, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5913277268409729, 'Além da caminhada, você pode fazer treinos na esteira,\nds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.5429662466049194}
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Como é uma alimentação saudavel?
{'A alimentação deve ser saudável e balanceada, estimulando a ingestão de água e fibras.': 0.6007276177406311, 'Para ingerir uma dieta variada, além da \ndisponibilidade dos alimentos, é fundamental a formação dos hábitos \nalimentares.': 0.530763566493988, 'A formação de hábitos \nalimentares saudáveis é facilitada se a criança for "ensinada a gostar de \nalimentos de alto valor nutritivo.': 0.5541978478431702, 'O manejo da criança anoréxica deve fazer parte de qualquer \nprograma de educação nutricional que vise a promover uma alimentação \nadequada para crianças pequenas.': 0.49889814853668213, 'São ricos em cálcio e, por isso, devem fazer parte de uma alimentação saudável.': 0.5151442289352417, ' Só uma alimentação variada evita a monotonia da dieta e \ngarante a quantidade de ferro e vitaminas que a criança \nnecessita, mantendo uma boa saúde e crescimento adequados.': 0.5865398645401001, ' Recomendações nutricionais\nManter a dieta equilibrada é fundamental para a manutenção de um bom\nestado nutricional, para o retorno do peso corporal, bom funcionamento\nintestinal e uma adequada produção deleite.': 0.5191260576248169, 'Recomendações nutricionais\nManter a dieta equilibrada é fundamental para a manutenção de um bom\nestado nutricional, para o retorno do peso corporal, bom funcionamento\nintestinal e uma adequada produção deleite.': 0.5191260576248169, ' Esses alimentos não são bons para a nutrição da criança e \ncompetem com alimentos mais nutritivos.': 0.5083054900169373, 'O aleitamento materno é a melhor e mais eficiente recomendação \ndietética para a saúde das crianças pequenas, tanto nas saudáveis quanto nas \ndoentes.': 0.5945478677749634}
A alimentação deve ser saudável e balanceada, estimulando a ingestão de água e fibras.
---------------------------------------------------
input Qual sua opinião sobre cereais?
{'O profissional de saúde deve avaliar se os mamilos estão invertidos ou não, o que vai ditar as orientações e informações \nque ele precisa fornecer à gestante quanto aos cuidados com as mamas e a amamentação.': 0.438512921333313, 'No \nNordeste, embora as mães tenham essa mesma opinião, elas não reconhecem \nesses alimentos como fontes de vitamina A. Os dados de todas as regiões \nsugerem que esses alimentos não são prioridades de compra e nem de \nconsumo.': 0.4974179267883301, 'A Pesquisa Qualitativa Nacional sobre Práticas Alimentares dos \nmenores de dois anos (Brasil, 1998a) indicou que existem algumas \npercepções das mães que podem estar influenciando o consumo de \nalimentos fontes de ferro.': 0.46750408411026, 'O Estudo Qualitativo Nacional de Práticas Alimentares (Brasil, 1998a) mostrou que, na Região Sudeste, 1/3 das entrevistadas (principalmente as que tem filhos entre seis e doze meses) não sabiam quais alimentos que fazem mal e outras responderam que não existe comida que faz mal, ou apontaram alguma situação em que a comida pode fazer mal, como comida pesada à noite.': 0.4508572816848755, 'O Estudo Qualitativo Nacional de Práticas Alimentares (Brasil, 1998a) \nindicou que não há um consenso, entre as mães entrevistadas, quanto à época de \n\nintrodução de alimentos complementares.': 0.472589373588562, 'O Estudo Qualitativo Nacional de Práticas Alimentares (Brasil, \n1998a) observou, nas entrevistas, que existe por parte das mães a identificação \nde alimentos “bons para a saúde”, como legumes, verduras e frutas.': 0.45436787605285645, 'É importante também no aconselhamento da escolha dos alimentos \nter conhecimento dos alimentos reconhecidos pelas mães como \naconselháveis para a alimentação das crianças pequenas.': 0.45155400037765503, 'Esse é um \naspecto importante a ser considerado na discussão com a mãe sobre a \nalimentação de seu filho.': 0.44243335723876953, 'Na Região Sul, observase que os alimentos considerados bons são \naqueles que sustentam, as comidas de sal.': 0.4415287971496582, 'Há também referência a \nalimentos frescos como alimentos bons.': 0.4842216968536377}
No 
Nordeste, embora as mães tenham essa mesma opinião, elas não reconhecem 
esses alimentos como fontes de vitamina A. Os dados de todas as regiões 
sugerem que esses alimentos não são prioridades de compra e nem de 
consumo.
---------------------------------------------------
input Arroz e massas fazem bem?
{'A mistura de arroz com feijão é e massas.': 0.7203351259231567, 'No Sudeste, frutas à tarde e feijão à \nnoite fazem mal.': 0.5963698625564575, 'A \nmistura de arroz com feijão, por exemplo, fornece proteínas de excelente \nqualidade, comparável com as da carne (Cameron e Hofvander, 1983).': 0.6213319301605225, 'Fazem parte deste grupo feijão,\nPrefira as versões soja, lentilha, ervilha e grão de bico.': 0.5281334519386292, 'Fazem parte deste grupo feijão, Prefira as versões soja, lentilha, ervilha e grão de bico.': 0.5281334519386292, 'O \nconsumo de outros alimentos varia de acordo com o local, mas quase \nsempre inclui biscoito, arroz, feijão, macarrão e batata.': 0.5613164901733398, 'Na Região Sudeste, foram considerados bons para a saúde as \ncarnes, principalmente de alguns tipos de peixe (parati, cará), frango (carne \nbranca, fígado), legumes, verduras, arroz e feijão.': 0.5310472846031189, 'Em todas as regiões, o leite engrossado com \nfarinhas, as sopas e o caldo de feijão aparecem como preferenciais. "Na Região \nSul, aparece ainda a polenta de consistência pastosa/mole."': 0.5396820306777954, ' Esses alimentos não são bons para a nutrição da criança e \ncompetem com alimentos mais nutritivos.': 0.5458047389984131, 'Por “preparação” entendese os alimentos elaborados com \nmodificação na sua estrutura – como, carne cozida, frango assado, etc, ou \nalimentos reunidos como em uma salada – por exemplo. de batata com \nalface e tomate.': 0.5437242388725281}
A mistura de arroz com feijão é e massas.
---------------------------------------------------
input Qual o tempo ideal para começar a pratica de exercicios na gestação?
{'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.772724986076355, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.772724986076355, ' Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.7773404121398926, 'Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.7773404121398926, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6354389786720276, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6512464284896851, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6512464284896851, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.6511438488960266, 'As atividades físicas mais adequadas são as \ncaminhadas, a natação e a hidroginástica.': 0.6443149447441101}
Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.
---------------------------------------------------
input Quando devo interromper os exercicios?
{'Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.561344563961029, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5198652744293213, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5198652744293213, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6167477369308472, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5151523351669312, ' Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.49812889099121094, 'Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.49812889099121094, 'A OMS recomenda a prática de exercícios moderados diariamente, por 30 minutos.': 0.5142855048179626, 'Evite exercícios entre 10 e 15 horas \nao ar livre.': 0.4965522289276123}
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Quais sao os treinos na gestação?
{'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.5887885689735413, ' Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.5601785778999329, 'Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.5601785778999329, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.6116567850112915, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.6116567850112915, 'Você poderá realizar todos os exercícios orientados paras gestantes.': 0.5592765808105469, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5939382910728455, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5939382910728455, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5888979434967041}
Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.
---------------------------------------------------
input Sobre exercicios com a vagina?
{'Como fazer os exercícios perineais\nContraia a vagina e o ânus, como se estivesse puxandoos para dentro e para cima do seu corpo.': 0.7674564719200134, 'Como fazer os exercícios perineais\nContraia a vagina e o ânus, como se estivesse puxandoos para dentro e para cima do seu corpo.Tente manter essa contração inicialmente por 3 segundos e em seguida solte os músculos devagar, até que esteja totalmente relaxada.\nDepois que estiver treinada, você pode manter a contração por até 10 segundos.\nProcure contrair o períneo sem usar outros músculos, como os do abdômen, as pernas ou as nádegas.\n"Respire normalmente durante a realização dos exercícios perineais. é muito importante que você não prenda\narespiração."\nSe você estiver com dificuldades em reconhecer quais músculos deve exercitar, faça o exercício de interromper o fluxo da urina, quando estiver no banheiro.\nNo entanto, não faça os exercícios enquanto estiver urinando.\n"Quando você estiver acostumada com os exercícios, alterne 10 contrações lentas (contraisegurarelaxa)\ncom uma série de 10 apertões rápidos(contraire soltar), depois repita o exercício, mas lentamente."': 0.6936852931976318, 'Como fazer os exercícios perineais\nContraia a vagina e o ânus, como se estivesse puxandoos para dentro e para cima do seu corpo.Tente manter essa contração inicialmente por 3 segundos e em seguida solte os músculos devagar, até que sinta que está totalmente relaxada."Depois que estiver treinada, você pode manter a contração por até 10 segundos. Procure contrair o períneo sem usar outros músculos, como os do abdômen, as pernas ou as nádegas. "Respire normalmente durante a realização dos exercícios perineais. é muito importante que você não prenda a respiração.""No entanto, não faça os exercícios enquanto estiver urinando."': 0.6864417195320129, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6174191236495972, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6174191236495972, ' Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.6299068331718445, 'Prática de exercício físico no puerpério. Pergunte ao seu médico quando poderá iniciar a prática de exercício físicoapós o nascimento do bebê.Comece devagar.': 0.6299068331718445, 'Depois que estiver treinada, você pode manter a contração\npor até 10 segundos.': 0.6245276927947998, 'É imperativo o diagnóstico de mamilos invertidos ou pouco desenvolvidos, e desde esse \nmomento Pode-se orientar a paciente a exercitar a expressão para facilitar o momento da \namamentação.': 0.63736891746521, 'Até a 12a semana de gestação é utilizada a técnica transvaginal, e, a partir dessa idade \ngestacional, a técnica transabdominal.': 0.6209412217140198}
Como fazer os exercícios perineais
Contraia a vagina e o ânus, como se estivesse puxandoos para dentro e para cima do seu corpo.
---------------------------------------------------
input Até que idade posso ter filho?
{'A média de idade gestacional de nascimento nas gestações \nduplas é de 36 semanas, enquanto nas gestações triplas é de 32 semanas.': 0.5570602416992188, 'A prevalência do aleitamento materno é baixa, sua duração é curta e o \naleitamento materno exclusivo até o sexto mês de vida é raro.': 0.6013776659965515, 'Após 6 meses a 1 ano do nascimento do bebê, o esperado é que você retorne\nao peso quetinha antes de engravidar.': 0.5745092630386353, 'Portanto a recomendação anterior da OMS sobre a duração \ndo aleitamento materno exclusivo de quatro a seis meses, fica substituída \npela recomendação acima, ou seja de duração do aleitamento materno \nexclusivo nos primeiros seis meses de vida.': 0.5511661171913147, 'Existe consenso de que a mulher que amamenta exclusivamente, em \nlivre demanda, até os seis meses após o parto e cuja menstruação ainda não \ntenha retornado, apresenta proteção contra nova gravidez (Family Health \nInternational, 1988).': 0.5683164596557617, 'A mulher vacinada deve aguardar quatro semanas para iniciar nova gestação.': 0.5786547660827637, '3.2.5 Duração do Aleitamento Materno \n \n O período natural de amamentação (sem a influência da cultura), \nsegundo diversas teorias, seria de 2,5 a sete anos.': 0.5563557147979736, ' Período de reserva proteica iniciase no segundo trimestre e estendese até o fim da \ngestação.': 0.559949517250061, '“A Gravidez é o melhor período para\nvocê adquirir hábitos saudáveis e leválos\npara sua vida toda!”\n \nOrientações para o puerpério\nO período após o parto chamase puerpério, também conhecido como pós\nparto.': 0.562529981136322}
A prevalência do aleitamento materno é baixa, sua duração é curta e o 
aleitamento materno exclusivo até o sexto mês de vida é raro.
---------------------------------------------------
input Como posso confirmar se estou gravida
{'Se for detectada incompatibilidade sanguínea entre a mãe e o feto, ou se houver suspeita dessa \nincompatibilidade, o médico deve iniciar imediatamente procedimentos para que a gestação \nse desenvolva de forma tranquila, evitando problemas maiores no futuro.': 0.4608117341995239, 'Sim, desde que você esteja realizando o pré-natal e não apresente nenhuma\ncomplicação na gravidez.': 0.5088653564453125, 'Nas situações em que a gestante nega ter seu cartão de vacinação \n(ou documentação válida que comprove vacinação), orientála sobre os momentos mais \nadequados para começar o processo vacinal.': 0.43095505237579346, 'Caso essa situação não seja possível, a gestante deve ser rastreada quanto a sua imunidade \ncontra a rubéola no início do pré-natal.': 0.44591575860977173, 'A mulher vacinada deve aguardar quatro semanas para iniciar nova gestação.': 0.4262722134590149, 'Deve evitar também ser radiografada, passando por esse procedimento apenas quando imprescindível, protegendo os órgãos reprodutores \ne informando aos técnicos de radiologia que está querendo engravidar ou está grávida.': 0.41031593084335327, 'Em casos de gestantes Rh negativo com parceiro Rh positivo, ou em que não se tem essa informação, \ndeve-se solicitar o Coombs indireto.': 0.4862961769104004, 'Além de ser um direito garantido por lei, as mulheres soropositivas podem ter uma gravidez \ntranquila, segura e com baixo risco de que seu filho nasça infectado pelo HIV, caso faça o \ncorreto acompanhamento médico e siga todas as recomendações e medidas preventivas.': 0.46394532918930054, 'As informações obtidas (idade, estado civil, profissão, ocupação, hábitos, nacionalidade, \nnaturalidade, procedência, razão da consulta, antecedentes pessoais, familiares, ciclos menstruais, \ndados da gestação atual e das gestações anteriores, se houver) e toda a evolução do pré-natal, \nincluindo resultados de exames, deverão ser registradas na ficha de pré-natal (documento do \nmédico) e no cartão da gestante (documento que fica em poder da paciente), documento de \nfundamental importância nas consultas de emergência ou na internação por ocasião do parto.': 0.4230737090110779, 'À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.': 0.4155588150024414}
Sim, desde que você esteja realizando o pré-natal e não apresente nenhuma
complicação na gravidez.
---------------------------------------------------
input Posso perder o bebe?
{'A mãe que tem \no vírus não deve amamentar o bebê, porque há risco de transmissão da mãe para o filho.': 0.5442133545875549, 'Caso essa situação não seja possível, a gestante deve ser rastreada quanto a sua imunidade \ncontra a rubéola no início do pré-natal.': 0.4552576541900635, 'A empregada deve requerer a licençamaternidade até o final do primeiro mês após o parto e o benefício também se \naplica à empregada que adotar ou obter guarda judicial por fins de adoção de criança.': 0.471987783908844, 'Placenta prévia e descolamento prematuro de placenta\nSituações que acarretam diversas complicações para o binômio maternofetal e que são \nresponsáveis também pelo nascimento de prematuros.': 0.48036277294158936, 'Dessa forma, a prática de atividade sexual pode ser interrompida em casos de ameaça de aborto, \nsangramento vaginal, ameaça de parto prematuro, placenta prévia ou insuficiência cervical.': 0.4698958396911621, 'Também é fundamental o suporte de uma boa maternidade, devido ao grande risco \nde prematuridade.': 0.47933512926101685, 'Caso engravide descompensada, a paciente merecerá reavaliação quanto à necessidade de prevenir o abortamento por insuficiência lútea, o que poderá \nocorrer e não é raro em mulheres com doenças da tireoide.': 0.5119956135749817, 'Gestantes com IgG negativo são \nsuscetíveis e, portanto, devem ser vacinadas após o parto.': 0.4686700105667114, 'Ocorre \nprincipalmente na ruptura prematura de membranas, mas pode ocorrer em até 20% das \ngestações com bolsa íntegra.': 0.455460786819458, 'Entretanto, há maior risco de complicações nutricionais, tanto para a gestante quanto \npara o feto.': 0.48952895402908325}
A mãe que tem 
o vírus não deve amamentar o bebê, porque há risco de transmissão da mãe para o filho.
---------------------------------------------------
input E sobre intervençoes?
{'Não parece comprometer o bemestar fetal, embora o desenvolvimento fetal intrauterino \ndemande acompanhamento cauteloso.': 0.4011019468307495, 'Entre o nascimento e a última procedência, pode haver uma gama de informações relevantes, principalmente no que se refere às infecções e infestações.Ainda nesse item deve-se questionar o eventual tipo de assistência pré-natal recebida até o momento e as intervenções que já foram iniciadas.': 0.49404311180114746, 'deve-se questionar a paciente sobre o uso de medicamentos prescritos ou não, e também sobre a \nrealização dos exames subsidiários.': 0.45002424716949463, 'Ainda não se sabe se experiências intrauterinas podem afetar futuras \npreferências de sabores.': 0.4041144847869873, 'Saber sobre as parcerias sexuais pode ser um indicativo da necessidade de aprofundar os questionamentos sobre doenças transmitidas por meio do ato sexual e os tratamentos realizados.': 0.42787009477615356, 'Informações sobre os diversos aparelhos e sistemas\nNesse item questionase a gestante sobre o funcionamento dos diversos aparelhos, \nrelacionando eventuais queixas às modificações e adequações fisiológicas à gravidez.': 0.4486570358276367, 'A legislação e o marketing de produtos que interferem na \namamentação um guia para o profissional de saúde.': 0.4257517457008362, 'Nos casos de gestação repetida, questionar também sobre a evolução do puerpério, \ntentando identificar informações sobre depressão puerperal, infecções e a experiência com \na amamentação.': 0.4274064302444458, 'Envolve o conhecimento de fatores potencialmente de risco, orientações e apoio a eventos menos favoráveis do que na gravidez, os quais, muitas \nvezes, são referidos ou evidenciados no contato inicial.': 0.4222707748413086, 'De forma geral, temos insistido no fato de que a gravidez deve, sempre que possível, ser pre\ncedida de consultas preconcepcionais.': 0.4083160161972046}
Entre o nascimento e a última procedência, pode haver uma gama de informações relevantes, principalmente no que se refere às infecções e infestações.Ainda nesse item deve-se questionar o eventual tipo de assistência pré-natal recebida até o momento e as intervenções que já foram iniciadas.
---------------------------------------------------
input Atendimentos mensais  no pre natal?
{'Cuidado Recomendado\npré-natal Atendimentos mensais até a 28ª semana\nAtendimentos quinzenais da 28ª até a 36ª semana\nAtendimentos semanais da 36ª semana até o parto e nascimento\nPuerpério Primeira consulta até o 7º dia pósparto\nSegunda consulta entre o 30º e 40º dias pósparto Os atendimentos subsequentes devem seguir a um roteiro similar ao da primeira consulta, porém mais simplificado com sempre maior atenção para os múltiplos fatores que podem determinar o desenvolvimento da gravidez.': 0.6008584499359131, 'A assistência pré-natal tem como objetivo prevenir os resultados indesejáveis (morbimortalidade \nmaterna e fetal) e, para isso, as consultas médicas devem ocorrer mais amiúde no primeiro \ntrimestre, devem ocorrer a cada 15 dias, no segundo trimestre todo mês, e no terceiro trimestre \na cada 15 dias, até 37a semana e, depois, semanalmente até a 40a semana.': 0.5026056170463562, 'A gestação costuma ser dividida em semanas, portanto até a 34a as consultas deverão ser mensais. depois, até a 36a semana, deverão ser quinzenais e, então, semanais até o momento do parto.': 0.511452317237854, 'As consultas prénatais devem ser agendadas a cada 3 a 4 semanas de intervalo até a 30a semana. a cada duas entre a 30a e a 34a semanas.e semanalmente após a 34a semana."Esses \nintervalos podem ser reduzidos em qualquer época da gravidez, de acordo com a necessidade."': 0.5307433009147644, 'O acompanhamento pré-natal ideal, conforme já referido, deve contemplar no mínimo \nseis consultas.': 0.49879324436187744, 'Pensando uma situação ideal para o calendário das consultas de pré-natal para gestantes sem \nrisco reprodutivo adicional, é razoável manter retornos mensais até a 32a semana, quinzenais \nda 32a até a 36a, e semanais até o nascimento.': 0.5739260911941528, 'Administrativamente, as consultas prénatais se dividem em primeira consulta (também \nchamada de caso novo de pré-natal) e os retornos.': 0.5468589663505554, 'A periodicidade e a frequência de agendamentos deve ser ditada pelo plano de cuidados, sendo mais intensa para \nas gestantes com instabilidade clínica.': 0.5020076036453247, 'Nesse primeiro contato, devem ser ainda discutidos número e frequência das consultas pré\nnatais. rotina das consultas (exames realizados a cada consulta). orientação sobre como localizar \no médico fora do horário comercial ou o serviço de emergência. responsabilidades da gestante \ne expectativas com relação à gestação e ao parto. agendamento das próximas consultas.': 0.4729230999946594, '\tMamadas frequentes, sem horários préestabelecidos (livre demanda).': 0.5524299144744873}
Cuidado Recomendado
pré-natal Atendimentos mensais até a 28ª semana
Atendimentos quinzenais da 28ª até a 36ª semana
Atendimentos semanais da 36ª semana até o parto e nascimento
Puerpério Primeira consulta até o 7º dia pósparto
Segunda consulta entre o 30º e 40º dias pósparto Os atendimentos subsequentes devem seguir a um roteiro similar ao da primeira consulta, porém mais simplificado com sempre maior atenção para os múltiplos fatores que podem determinar o desenvolvimento da gravidez.
---------------------------------------------------
input E sobre aplicativos para idade gestacional? eles funcionam?
{'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.4558577537536621, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.4558577537536621, 'O uso de medicamentos durante a gravidez é indicado para quem já está fazendo o tratamento \ne para a grávida que tem HIV, não apresenta sintomas e não está tomando remédios para \nAids.': 0.46656620502471924, 'O PregnosticonPlanotest é um dos mais antigos testes de gravidez que representam inibição da aglutinação.': 0.44600552320480347, 'Encefalite japonesa Não existem estudos que indiquem uso seguro dessa vacina durante o período gestacional.': 0.4678493142127991, '10 – A mamadeira é amplamente utilizada, mesmo em crianças \npequenas amamentadas.': 0.4619324207305908, 'Não há dúvidas quanto às vantagens da amamentação exclusiva \nnos primeiros meses de vida da criança e quanto às desvantagens \nda introdução precoce de outros alimentos e mesmo líquidos, \ncomo água ou chás, e do uso de mamadeiras/chupetas.': 0.467828631401062, 'Mechas, luzes e o uso de hena \nestão liberadas pelos obstetras após a primeira metade da gestação.': 0.4491361975669861, 'O uso do ácido fólico na préconcepção costuma ser recomendado para qualquer mulher que pretenda engravidar, como já foi exposto .': 0.46181797981262207}
Encefalite japonesa Não existem estudos que indiquem uso seguro dessa vacina durante o período gestacional.
---------------------------------------------------
input E sobre vacinas, quais tenho q fazer?
{'Se estiver indicada vacinação, Devem-se administrar as vacinas prontamente.': 0.47241467237472534, 'Orientações sobre as vacinas e imunizações\nNesse momento sentese o valor das orientações préconceptivas, em que as providências \ncom relação à imunização são tomadas. "Como na maioria dos casos isso não ocorre, \ndevemos estar preparados para orientar a paciente grávida sobre as vacinas necessárias."': 0.48497021198272705, 'Com esquema de vacinação básico incompleto (que tenha recebido menos de \nTríplice bacteriana (DTPa) três doses do componente tetânico durante a vida) uma dose de dTpa seguida \npor uma ou duas doses de dT para completar o esquema de três doses."Reforço \na cada dez anos com dT ou dTpa.""Fazer dTpa no puerpério, se não vacinada \ndurante a gestação, ou dT seis meses após a última dose recebida na gravidez."': 0.5008689165115356, 'Mulheres grávidas que receberam a vacina ou que engravidaram em menos de \nquatro semanas após a administração devem ser orientadas quanto ao risco e acompanhadas \nno pré-natal.': 0.46263647079467773, 'Uma ou duas doses (com intervalo mínimo de trinta dias) para mulheres nascidas Tríplice viral (sarampo, após 1962, de acordo com o histórico vacinal, de forma que todas recebam no caxumba, rubéola) mínimo duas doses na vida.': 0.4674803614616394, 'Quando há necessidade de proteger a gestante contra a coqueluche, recomenda-se, por prudência, administrar a vacina entre a 27a e a 36a semana, conquanto estudos afirmem que pode ser administrada em qualquer idade gestacional.': 0.4712878465652466, 'Antipneumocócica (PCV 13 e PPV23) Até o momento, não existem evidências de que se deve indicar a vacina conjugada pneumocócica às gestantes.': 0.4617716073989868, 'A exceção está nas gestantes com problemas imunológicos, que devem receber medicações específicas \naté o 7o mês (heparina, aspirina, corticoides etc.).': 0.5019479990005493, 'Se tiver sido vacinada há mais de dez anos, poderá receber até três doses com intervalo de um mês, dependendo do passado vacinal (caso não tenha completado as três doses de antitetânica previamente à gravidez).16 Apesar de não haver evidências de danos fetais causados pelos toxoides tetânico e diftérico, recomenda-se que a aplicação da primeira dose ou reforço ocorra a partir do segundo trimestre, para evitar eventuais danos.': 0.4693032503128052, 'Apesar de ser pouco estudada, o uso de imunoglobulina \nantivaricela (VZIG) é indicada a todas as gestantes suscetíveis e que tenham sido expostas.': 0.4789034128189087}
A exceção está nas gestantes com problemas imunológicos, que devem receber medicações específicas 
até o 7o mês (heparina, aspirina, corticoides etc.).
---------------------------------------------------
input Tenho que fazer a vacina da febre amarela?
{'Caso a gestante não tenha sido imunizada durante a infância e apresente alto risco de adquirir a doença, a vacinação pode ser realizada de acordo com as recomendações de vacinação em adultos.': 0.6698669195175171, 'A vacina contra tétano deve ser administrada de acordo com a carteira de vacinação.': 0.7005553841590881, 'Febre tifoide Não existem dados sobre o uso da vacina anti tifoide em gestantes.': 0.6838092803955078, 'Vacinação \nA mulher deve tomar vacina contra sarampo, caxumba e rubéola (MMR), caso não seja \nimune a essas doenças, e deve ser orientada a não engravidar nos três meses seguintes.': 0.730191171169281, 'Rubéola\nTodas as mulheres em período reprodutivo deveriam ser rastreadas para a rubéola, e as mais \nsuscetíveis deveriam ser vacinadas antes da gestação, uma vez que não há tratamento que \nprevina a transmissão vertical.': 0.689141035079956, 'Gestantes com IgG negativo são \nsuscetíveis e, portanto, devem ser vacinadas após o parto.': 0.6497229933738708, 'Para a indicação da dTpa na gestante, não importa o intervalo de tempo desde a última dose de dT, posto que é essa vacina que vai proteger o recém-nascido contra a coqueluche, até que seu esquema vacinal esteja completo.': 0.7016062140464783, 'Antipneumocócica (PCV 13 e PPV23) Até o momento, não existem evidências de que se deve indicar a vacina conjugada pneumocócica às gestantes.': 0.6635320782661438, 'Também deve-se vacinar contra a hepatite B durante a gestação se o antígeno for obtido \nda cápsula viral.': 0.7146657705307007, 'A Hepatites A, B ou A + B vacinação combinada contra as hepatites A e B é uma opção e pode substituir a \nvacinação isolada contra as hepatites A e B.\nCom esquema de vacinação básico completo reforço com dTpa (tríplice bac\nteriana acelular do tipo adulto) ou dT (dupla do tipo adulto) a cada dez anos.': 0.6370460987091064}
Vacinação 
A mulher deve tomar vacina contra sarampo, caxumba e rubéola (MMR), caso não seja 
imune a essas doenças, e deve ser orientada a não engravidar nos três meses seguintes.
---------------------------------------------------
input Tenho HIV, quais os problemas disso?
{'Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.': 0.5401490330696106, 'Mulheres que apresentam risco de adquirir hepatite B (mulheres com múltiplos parceiros \nou cujo parceiro seja HbsAg positivo, portadoras de doenças sexualmente transmissíveis, \nusuárias de drogas endovenosas) devem ser vacinadas.2,7\nHPV (papilomavírus humano) Vacina não recomendada na gestação, ainda que não \nexistam estudos que demonstrem riscos fetais.': 0.518352746963501, 'b) Pesquisa de infecções\n Viroses Investigar as viroses no período pré-concepcional nos dá informações sobre a situação da doença se é imune, susceptível ou ativa.': 0.5171264410018921, 'Diferentes técnicas são descritas na prática e podem conduzir a problemas disab\nsortivos, como deficiência de várias vitaminas, herniações, torções intestinais, entre outras \nintercorrências.': 0.5037283301353455, 'Além disso, grande parte das infecções fetais ocorre por reativação em mulheres previamente expostas e, dessa forma, pacientes com IgG positivo não se isentam AntiHCV (hepatite C) O rastreamento para o HCV reservase às gestantes de risco, que são as usuárias de drogas injetáveis, presidiárias, mulheres HIV positivas ou com parceiros HIV positivos, mulheres expostas a hemoderivados ou que receberam transfusão de sangue, bem como gestantes com alteração da função hepática, com múltiplos parceiros ou tatuadas.': 0.5187574028968811, 'ESTADO CIVIL\nAs gestantes sem parceiro fixo têm mais problemas emocionais, sexuais e socioeconômicos.': 0.5032253861427307, 'ANTECEDENTES FAleitamento MaternoILIARES\nDevem-se investigar doenças familiares de caráter hereditário que possam aumentar a \nmorbimortalidadade maternofetal, tais como diabetes, hipertensão, préeclâmpsia/\neclâmpsia, doenças autoimunes, trombose, cardiopatias congênitas, malformações, \ngemelaridade, doenças infecciosas (tuberculoses, hepatites B e C, HIV e sífilis), em particular \nquando atingem parentes de primeiro grau da gestante."No que diz respeito ao parceiro, \né necessário investigar gemelaridade, doenças infecciosas e sexualmente transmissíveis, \nmalformações e alcoolismo (fator de risco para malformações)."\n"Em pacientes negras, é maior a prevalência de hipertensão arterial, miomatose uterina, \nanemia falciforme e vícios pélvicos."\n"deve-se fazer rastreamento para a doença de TaySachs \nentre judeus e de talassemias em mulheres de origem mediterrânea."': 0.5157088041305542, 'Antecedentes familiares\nEntre os antecedentes familiares de realce, deve-se tentar identificar se existem familiares hipertensos, diabéticos, com doenças autoimunes, cardiopatas, parentes de primeiro grau que apresentaram préeclâmpsia/eclâmpsia, gemelaridade, distócias no parto e doenças infectocontagiosas (tuberculose, hepatites B e C, vírus da imunodeficiência humana, sífilis, toxoplasmose, rubéola, herpes, esquistossomose, entre outras).': 0.5066666007041931, 'De acordo com a American Society for Reproductive Medicine, aproximadamente 1/3 das \nmulheres entre 35 e 39 anos e 2/3 das mulheres acima de 40 anos apresentam dificuldade \npara engravidar, sendo a maioria dos casos de infertilidade tratados com sucesso.': 0.529791533946991, 'TSH\nA pesquisa de doenças da tireoide no pré-natal justificase por acometerem mulheres em \nsua fase reprodutiva.': 0.5272173881530762}
Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.
---------------------------------------------------
input Quais são os exames que preciso fazer?
{'deve-se explicar à gestante que alguns exames serão realizados logo após a primeira consulta pré-natal e outros em momentos distintos da \ngravidez.': 0.4572257995605469, 'Em relação aos exames laboratoriais de rotina, estes são solicitados de maneira semelhante ao \nque ocorre nas gestações de feto único, sem necessidade de agregar outros pedidos.': 0.4622492790222168, 'Na avaliação inicial da grávida, deverão ser discutidos seu estado geral e os exames \nlaboratoriais a serem solicitados.': 0.4806034564971924, 'Outros exames, como eletrocardiograma, devem ser solicitados de acordo com a avaliação clínica obstétrica da gestante.': 0.4692087173461914, 'Muitos exames que foram realizados no início da avaliação para o tratamento de \nreprodução assistida não terão de ser repetidos, a não ser que haja dúvidas.': 0.45209741592407227, 'PREVENÇÃO\nNa primeira consulta do pré-natal, a gestante deve fazer um exame de sangue que \ndetermine o seu grupo de sangue (ABO), o tipo de RH e a pesquisa de anticorpos.': 0.4421062469482422, 'A citologia oncótica deve ser realizada se a \ngestante não a tiver feito recentemente.': 0.4687572121620178, 'O exame deve ser realizado por volta de 35 semanas, \npara que se promova a profilaxia intraparto.': 0.45010244846343994, 'Exame físico ginecoobstétrico\nEm termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal \ndeverá considerar o exame das mamas e o exame obstétrico propriamente dito.': 0.45903128385543823, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.4767187833786011}
Na avaliação inicial da grávida, deverão ser discutidos seu estado geral e os exames 
laboratoriais a serem solicitados.
---------------------------------------------------
input De quanto em quanto tempo devo ir consulta
{'Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.4788486361503601, 'As consultas prénatais devem ser agendadas a cada 3 a 4 semanas de intervalo até a 30a semana. a cada duas entre a 30a e a 34a semanas.e semanalmente após a 34a semana."Esses \nintervalos podem ser reduzidos em qualquer época da gravidez, de acordo com a necessidade."': 0.409912109375, 'Orienta-se que essa consulta seja agendada para no máximo 15 dias após a inicial, mas essa proposta de cronograma pode não ser adequada para todas as gestantes classificadas como sendo de risco \nreprodutivo, pois pode demandar uma frequência de retornos na dependência da condição mórbida associada ou intercorrente.': 0.39616984128952026, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.40708088874816895, 'A segunda consulta puerperal ocorre entre 30 e 40 dias pósparto.': 0.4022836685180664, 'Nesse primeiro contato, devem ser ainda discutidos número e frequência das consultas pré\nnatais. rotina das consultas (exames realizados a cada consulta). orientação sobre como localizar \no médico fora do horário comercial ou o serviço de emergência. responsabilidades da gestante \ne expectativas com relação à gestação e ao parto. agendamento das próximas consultas.': 0.39871662855148315, 'Pensando uma situação ideal para o calendário das consultas de pré-natal para gestantes sem \nrisco reprodutivo adicional, é razoável manter retornos mensais até a 32a semana, quinzenais \nda 32a até a 36a, e semanais até o nascimento.': 0.3880174160003662, 'Existem muitas dúvidas sobre o momento ideal para iniciar o pré-natal.Alguns autores acreditam que deveria ser o mais precoce possível, enquanto outros acham que deveria ocorrer no final do primeiro trimestre.': 0.42215871810913086, 'Depois disso, deve \nocorrer a cada dois ou três dias até a 41a semana.': 0.4063197374343872}
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input O que garante o pre-natal?
{'Após o início do pré-natal, todo esforço se justifica no sentido de permitir que o controle periódico, oportuno e \ncontínuo das grávidas seja assegurado.': 0.5215746164321899, 'Na realidade, a opção de iniciar precocemente o pré-natal é baseada nas possibilidades de acesso ao sistema de saúde.': 0.4839599132537842, 'Com intuito de atrair as gestantes o mais precocemente possível, o Ministério da Saúde tem estimulado as instituições com benefícios extras quando a grávida inicia \nrapidamente o pré-natal e garantese o mínimo de consultas, exames, vacinação e retorno no puerpério.': 0.5797672867774963, 'Estímulo da participação do parceiro ou dos familiares no pré-natal\ndeve-se dar ao parceiro da gestante a opção de participar do pré-natal."São inúmeras as \nvantagens dessa participação para a gestante, para o parceiro e para o futuro filho."': 0.4841533303260803, 'Portanto, a assistência pré-natal deve ser iniciada antes da concepção, \nde forma a garantir que a mulher esteja fisicamente apta a suportar essa sobrecarga.': 0.49880337715148926, 'Caso já tenha sido vacinada nos \núltimos dez anos, a mulher deverá receber apenas o reforço.': 0.5127406120300293, 'Em primeiro lugar, ela garante, em muitos casos, a \nsobrevivência das crianças, e, particularmente, daquelas em condições \ndesfavoráveis e/ou que nascem com baixo peso.': 0.49752306938171387, 'Nos dados da Região Nordeste, por exemplo, os relatos das mães \nindicam que o “leite materno deve ser dado como único alimento para as \ncrianças até seis meses porque sustenta e protege a criança contra doenças e \naumenta o amor da criança pela mãe”.': 0.48501789569854736, 'O teste para HIV é recomendada no primeiro trimestre e repetida no terceiro trimestre ou até \nna hora do parto.': 0.5002403259277344, 'Meningocócica conjugada Uma dose, mesmo para aquelas vacinadas na infância ou há mais de cinco anos.': 0.48853015899658203}
Com intuito de atrair as gestantes o mais precocemente possível, o Ministério da Saúde tem estimulado as instituições com benefícios extras quando a grávida inicia 
rapidamente o pré-natal e garantese o mínimo de consultas, exames, vacinação e retorno no puerpério.
---------------------------------------------------
input Gostaria de saber sobre sindromes hipertensivas?
{'Doenças como diabetes, hiper ou hipotireoidismo podem ser suspeitadas ou informadas no questionário.': 0.616037905216217, 'Alterações vasculares retinianas não são comuns, exceto em patologias como síndromes hipertensivas ou diabetes.': 0.6024606227874756, 'Manifestações como hiperemese gravídica, enxaqueca e alguns distúrbios psiquiátricos (hipomania, \ndepressão) podem estar relacionadas a alterações vasculares e hormonais exclusivas da \ngravidez.': 0.5742523670196533, 'Doenças maternas\nTanto as doenças crônicas maternas, como diabetes mellitus e hipertensão arterial sistêmica, quanto as desenvolvidas na gestação com a préeclâmpsia estão envolvidas no trabalho de parto espontâneo, pela liberação de catecolaminas devido ao estresse materno e fetal, e no eletivo, sendo responsáveis por diversas complicações da mãe e do concepto.': 0.5484492778778076, 'Na literatura, é consenso que desnutrição, gravidez múltipla, anomalias cromossômicas, causas infecciosas, estados hipertensivos, diabetes não compensada, tabagismo, drogas, alcoolismo e altitude elevada são os principais fatores conhecidos na etiologia do CFR.': 0.5720652341842651, 'Assim, podemos citar a) Dados pessoais nome, endereço, idade, biotipo, estado civil, naturalidade, profissão.\nb) Queixa e duração (se houver).\nc) História pregressa da moléstia atual (se houver).\nd) Antecedentes pessoais doenças como hipertensão, endocrinopatias, cardiopatias, doenças sexualmente transmissíveis e outras. vacinação, hábitos e vícios. "\ne) Antecedentes familiares hipertensão arterial sistêmica, diabetes melito, cardiopatias congênitas, malformações, gestação gemelar, doenças transmissíveis, mastopatias etc.\nf) História fisiológica desde o nascimento. menarca, ciclos menstruais, atividade sexual (frequência, início, queixas).\ng) Antecedentes obstétricos (detalhar cada gestação).\nh) Atividade profissional (fatores de risco)."': 0.5752809643745422, 'Entre os antecedentes familiares, os principais são diabetes, hipertensão, cardiopatias e gemelaridade, principalmente da família materna.': 0.5825182199478149, 'Algumas doenças infectocontagiosas, reumatismos, nefropatias, diabetes, miocardiopatias \ncrônicas (doença de Chagas), uropatias, vulvovaginites, entre outras, podem reagudizar na \ngestação, trazendo riscos para o binômio mãefeto.': 0.5488255620002747, 'Sua indicação é para gestantes com doenças que possam levar à insuficiência placentária, tais como \nhipertensão arterial, diabetes mellitus, trombofilias, cardiopatias, lúpus eritematoso sistêmico \ne pneumonias.': 0.5782591700553894, 'Qualquer estratégia de prevenção de IH em UTI deve conter medidas específicas de prevenção de sepse primária associada a cateter vascular central.\nO diagnóstico das infecções no RN muitas vezes é difícil, uma vez que as manifestações clínicas são inespecíficas e podem ser confundidas com outras doenças próprias dessa faixa etária.': 0.5535076856613159}
Doenças como diabetes, hiper ou hipotireoidismo podem ser suspeitadas ou informadas no questionário.
---------------------------------------------------
input Como devo tratar a sifilis?
{'Alguns agentes infecciosos (rubéola, \ntoxoplasmose, citomegalovirose, sífilis, entre outros) podem provocar malformações fetais.': 0.5966827869415283, 'As gestantes que sabem da infecção durante o pré-natal têm indicação de \ntratamento com medicamentos para prevenir a transmissão para o feto.': 0.567904531955719, 'Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.': 0.5616971254348755, 'As principais viroses a serem pesquisadas são rubéola, doença de inclusão citomegálica, \nherpes simples, hepatites B e C, papilomavírus humano, influenza e HIV.': 0.6332756280899048, 'Tal situação não requer tratamento, o qual se reserva ao aparecimento de infecções causadas \npor cândida ou vaginose bacteriana.': 0.6061456203460693, 'A vacina antirrubéola por vírus atenuado é secretada no leite, mas há apenas relatos de infecção assintomática.': 0.5683634281158447, 'Antecedentes familiares\nEntre os antecedentes familiares de realce, deve-se tentar identificar se existem familiares hipertensos, diabéticos, com doenças autoimunes, cardiopatas, parentes de primeiro grau que apresentaram préeclâmpsia/eclâmpsia, gemelaridade, distócias no parto e doenças infectocontagiosas (tuberculose, hepatites B e C, vírus da imunodeficiência humana, sífilis, toxoplasmose, rubéola, herpes, esquistossomose, entre outras).': 0.563910722732544, 'Nem sempre é fácil distinguir a mastite infecciosa da não infecciosa apenas pelos sinais e \nsintomas.': 0.586902916431427, 'Citomegalovírus\nNão há consenso do rastreamento de rotina do citomegalovírus no pré-natal, uma vez que o \ndiagnóstico da doença aguda é extremamente difícil.': 0.5869102478027344, 'A prevenção do abscesso mamário está intimamente relacionada à prevenção e tratamento \nadequado e precoce da mastite.': 0.6040521264076233}
As principais viroses a serem pesquisadas são rubéola, doença de inclusão citomegálica, 
herpes simples, hepatites B e C, papilomavírus humano, influenza e HIV.
---------------------------------------------------
input Quais os melhores exercicios aerobicos?
{'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6043884754180908, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.6043884754180908, 'EXERCÍCIOS FÍSICOS\nA prática de exercícios aeróbicos regulares durante a gravidez parece melhorar ou manter a \ncapacidade física e a boa imagem corporal da gestante.': 0.5291231870651245, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5987403392791748, 'Os exercícios mais \naconselhados são caminhada, natação, hidroginástica, ioga e alongamento.': 0.5502864122390747, 'As atividades físicas mais adequadas são as \ncaminhadas, a natação e a hidroginástica.': 0.5421258211135864, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.582875669002533, 'Você poderá realizar todos os exercícios orientados paras gestantes.': 0.5832732915878296, 'Você poderá realizar todos os exercícios orientados paraas gestantes (Guia prático de exercícios).': 0.576645016670227, 'Atenção também deve ser dispensada à formação do mamilo invertido ou mesmo ao mamilo plano.Nesses casos, estão indicados os exercícios de Hoffman para forçar a extrusão mamilar.': 0.5284631252288818}
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Quanto tempo deve durar os exercicios?
{'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.5813498497009277, 'Esses períodos, em geral, duram de 2 a 3 dias e \ncostumam ocorrer entre 10 e 14 dias de vida, entre 4 e 6 semanas e em \ntorno dos 3 meses.': 0.64211106300354, 'Gestação prolongada\nA gestação tornase prolongada quando atinge duração superior a 294 dias ou completa 42 \nsemanas.': 0.5783128142356873, 'A duração mediana do Aleitamento Materno E foi de \n54 dias (0,7 a 89 dias) e a do Aleitamento Materno de 342 dias (293 a 601 dias).': 0.6435244083404541, ' Período de reserva proteica iniciase no segundo trimestre e estendese até o fim da \ngestação.': 0.5645388960838318, 'Seu uso deve ser iniciado um mês antes da gravidez e continuado até a 12a \nsemana gestacional.': 0.5479987263679504, 'O tempo recomendado de interrupção da amamentação varia dependendo da droga.': 0.5728254318237305, '3.2.5 Duração do Aleitamento Materno \n \n O período natural de amamentação (sem a influência da cultura), \nsegundo diversas teorias, seria de 2,5 a sete anos.': 0.5872121453285217, 'A duração mediana da amamentação no Brasil (período de tempo \nem que metade das crianças ainda está sendo amamentada), também vem \naumentando, passando de 2,4 meses em 1975 para 5,5 meses em 1989 \n(Venâncio, 1996).': 0.5498833656311035, 'Como a situação anterior pode corresponder ao período de sono fetal, deve-se continuar o \nexame por mais 20 minutos.': 0.555799126625061}
A duração mediana do Aleitamento Materno E foi de 
54 dias (0,7 a 89 dias) e a do Aleitamento Materno de 342 dias (293 a 601 dias).
---------------------------------------------------
input E bom beber liquidos ?
{'Beba líquidos em abundância, principalmente leite, suco de frutas e água,\npois a amamentação dá muita sede.': 0.686680257320404, 'Caminhadas, alongamentos,fortalecimentos e exercícios perineais.Beba bastante líquido durante o treino de exercício, pois a falta de hidrataçãopode prejudicar a produção deleite materno.': 0.5399277806282043, ' Com a introdução da alimentação complementar, é importante \nque a criança beba água nos intervalos das refeições.': 0.5364012718200684, ' Aleitamento materno predominante – quando o lactente recebe, \nalém do leite materno, água ou bebidas à base de água, como \nsucos de frutas e chás.': 0.5636217594146729, 'recomenda-se o consumo da fruta inteira em substituição ao suco, pois muitos nutrientes podem ser perdidos no preparo de sucos naturais.': 0.5499135851860046, 'recomenda-se o consumo da fruta inteira em substituição ao suco, pois muitos\nnutrientes podem ser perdidos no preparo de sucos naturais.Quanto mais variedade melhor.': 0.5450319051742554, 'No Sudeste, frutas à tarde e feijão à \nnoite fazem mal.': 0.5317858457565308, 'Portanto, para melhorar o \naproveitamento do ferro do alimento complementar, é válido recomendar a \nadição de carne bovina, peixe ou ave nas dietas, mesmo que seja em \npequena quantidade e a oferta, logo após as refeições, de frutas cítricas ou \nsucos com alto teor de ácido ascórbico (Derman et al., 1980. Gilloly et al., \n1983. Hallmberg et al., 1987. Dallman, 1990).': 0.5474424362182617}
Beba líquidos em abundância, principalmente leite, suco de frutas e água,
pois a amamentação dá muita sede.
---------------------------------------------------
input Posso comer castanhas e nozes?
{' gema, banana) não podem ser \ncomidos por mulheres, pois causam gestação gemelar.': 0.5058934092521667, ' “comida de animais” – Norte folhas verdes em geral. \nNordeste batata doce, folhas verdes em geral, jerimum/abóbora.': 0.5311036109924316, 'O \naçúcar somente deve ser usado na alimentação da criança após \num ano de idade.': 0.4793887138366699, 'Anemia  Comerdiariamente alimentos fontes de Ferro, como carne por vermelha, fígado, folhas verdes escuras e leguminosas\ncarência (feijão, soja, lentilha). "Consumir junto com os alimentos fontes de ferro, suco ou\nprevenir fruta cítrica (laranja, limão, acerola)."': 0.47711503505706787, 'Fazem parte deste grupo feijão,\nPrefira as versões soja, lentilha, ervilha e grão de bico.': 0.46791142225265503, 'Fazem parte deste grupo feijão, Prefira as versões soja, lentilha, ervilha e grão de bico.': 0.46791142225265503, 'A seguir, são apresentadas algumas misturas que, segundo a percepção das \nmães, fazem mal \n \n Norte – leite com manga, caju, goiaba, açaí. açaí com outras \nfrutas. peixe com carne.': 0.4694432020187378, ' CentroOeste – leite com manga, beterraba, ovo. ovo com \nmanga, abacaxi, peixe. maçã com banana.': 0.46459078788757324, 'No Nordeste (Piauí) as preferências de alimentos para crianças pequenas e as razões indicadas pelas mães foram acerola, rica em vitamina C, combate a gripe. beterraba e feijão – fonte de ferro. arroz, macarrão – fácil preparo e digestão. iogurte – alimento que tem status. leite de vaca – bom para a saúde, criança cresce forte, fácil de ser oferecido (por mamadeira, diversas preparações). e vegetais – alimentos suaves (Monte e Sá, 1998).': 0.45952630043029785, ' “proibidos pela religião” – Norte camarão, carne de porco.': 0.4803289771080017}
 “comida de animais” – Norte folhas verdes em geral. 
Nordeste batata doce, folhas verdes em geral, jerimum/abóbora.
---------------------------------------------------
input o que e bom evitar?
{'Por isso, deve-se \norientar a vida social de modo a evitar o consumo exagerado e continuado de álcool.': 0.5549786686897278, 'Cuidados para evitar ou reduzir o risco de cloasma\nPara evitar o cloasma, Orienta-se evitar a exposição solar entre 10 e 16 horas, e usar cremes \nfotoprotetores, chapéu, sombrinha ou guardasol.': 0.541577160358429, 'A paciente deve evitar ao máximo o uso de medicamentos sem indicação precisa, ficando atenta aos efeitos colaterais e \nteratogênicos durante a gravidez.': 0.5019112825393677, 'Orientações sobre evitar o decúbito horizontal logo após as refeições, assim como fracionar \na dieta, evitando a ingestão de alimentos como café, chá, chocolate e frituras, normalmente \nsão suficientes.': 0.5819111466407776, 'Sobre o uso de medicamentos, Orienta-se a gestante a não ingerilos sem prescrição médica.': 0.5059810280799866, 'Vale salientar que a orientação contraceptiva visando a evitar a gestação, que na grande maioria das vezes é indesejada, deve ser estimulada.': 0.5007227063179016, 'A conduta deve ser semelhante à preconizada quando da vacina contra sarampo.': 0.4997069835662842, 'Muitos recomendam que a mãe amamente a criança \nantes de oferecer a alimentação complementar, na tentativa de prevenir possível \nimpacto negativo da ingestão desses alimentos no volume de leite materno \nconsumido e produzido.': 0.5004576444625854, 'Orienta-se dieta fracionada com menor volume de alimento em alguns casos, sugerindo a \ningestão de substâncias pastosas.': 0.508988082408905}
Orientações sobre evitar o decúbito horizontal logo após as refeições, assim como fracionar 
a dieta, evitando a ingestão de alimentos como café, chá, chocolate e frituras, normalmente 
são suficientes.
---------------------------------------------------
input E importante relaxar?
{'É muito importante que após os exercícios você faça um relaxamento para se\nrecuperar e aumentar sua interação com o bebê.': 0.6960790157318115, 'Prático de Exercícios\nRelaxamento\nÉ muito importante que após os exercícios você faça um relaxamento para se\nrecuperar e aumentarsuainteração com o bebê.': 0.7069780826568604, 'A simples orientação de, nessas ocasiões, manterse deitada do lado esquerdo até o cessar \ndos sintomas, além de adotar uma dieta fracionada e manterse em ambientes arejados \ncostumam ser tratamentos efetivos.': 0.5737955570220947, 'Fique deitada de lado, com a mãona barriga, fazendo respiraçõeslongas e profundas\ne vá conversando com seu bebê.': 0.5298975110054016, 'Caminhadas, alongamentos,fortalecimentos e exercícios perineais.Beba bastante líquido durante o treino de exercício, pois a falta de hidrataçãopode prejudicar a produção deleite materno.': 0.5431910753250122, 'Evite se exercitar em ambientes sem ventilação e quentes.': 0.527490496635437, 'Seu tratamento consiste na orientação de repouso, com os membros superiores elevados e \nutilização de meias elásticas.': 0.5253657102584839}
Prático de Exercícios
Relaxamento
É muito importante que após os exercícios você faça um relaxamento para se
recuperar e aumentarsuainteração com o bebê.
---------------------------------------------------
input Tenho que realizar uma dieta na gestação?
{'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.4831674098968506, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.4678751230239868, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.4678751230239868, 'Nesse período, é recomendável uma dieta com mais \ncalorias e proteínas.': 0.4474438428878784, 'Se você já praticava exercício antes de engravidar e sua gestação está saudável, você pode continuar a fazer os exercícios, mas talvez precise fazer algumas adaptações,\ncomodiminuir o ritmo e/ou mudaralguns exercícios.Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.4514954686164856, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.45001399517059326, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.45001399517059326, 'Ainda nesses trimestres deve haver, além de atividade física orientada, dieta equilibrada em torno de 30 kcal/kg de peso, com hidratação abundante de mais ou menos 2 l/dia.': 0.4499618411064148, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.44873249530792236}
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Gravidez
{'“A Gravidez é o melhor período para\nvocê adquirir hábitos saudáveis e leválos\npara sua vida toda!”\n \nOrientações para o puerpério\nO período após o parto chamase puerpério, também conhecido como pós\nparto.': 0.5583873391151428, 'A amamentação exclusiva é importante também na diminuição da \nfertilidade após o parto.': 0.552008330821991, 'A mulher vacinada deve aguardar quatro semanas para iniciar nova gestação.': 0.6485320925712585, 'Existe consenso de que a mulher que amamenta exclusivamente, em \nlivre demanda, até os seis meses após o parto e cuja menstruação ainda não \ntenha retornado, apresenta proteção contra nova gravidez (Family Health \nInternational, 1988).': 0.606415331363678, 'Inicialmente, o organismo materno experimenta uma fase anabólica, que dura pouco mais da \nmetade da gestação.': 0.5295586585998535, ' Ovários\nA produção ovariana de progesterona se dá no corpo lúteo até a 7a semana de gestação, \na partir da qual a placenta assume sua autonomia hormonal.': 0.5503067970275879, ' Período de reserva proteica iniciase no segundo trimestre e estendese até o fim da \ngestação.': 0.5495696663856506, 'A prevalência do aleitamento materno é baixa, sua duração é curta e o \naleitamento materno exclusivo até o sexto mês de vida é raro.': 0.5428696870803833, 'À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.': 0.5798051953315735, 'Após esse período, iniciase a fase de catabolismo até o final da gestação.': 0.5441206693649292}
“A Gravidez é o melhor período para
você adquirir hábitos saudáveis e leválos
para sua vida toda!”
 
Orientações para o puerpério
O período após o parto chamase puerpério, também conhecido como pós
parto.
---------------------------------------------------
input Sobre omega 3?
{'Geralmente, numa primeira gestação, não há qualquer problema.': 0.44988298416137695, 'Por outro lado, não existem dados que demonstrem benefícios de sua utilização nesse período.': 0.46640706062316895, 'São poucos os dados referentes à época de introdução de alimentos \ncomplementares para crianças no País.': 0.45652925968170166, 'A laranja aparece \nnesse grupo apenas na Cidade de Curitiba.': 0.4606250524520874, '2 – A amamentação exclusiva ainda é pouco praticada em todas \nas regiões.': 0.4802435636520386, ' “carregados” – Sudeste sardinha, robalo.': 0.4658282995223999, '3 – É comum a introdução precoce dos alimentos complementares.': 0.4757486581802368, 'Em primeiro lugar, observase a predominância da oferta \nde água, chás e sucos desde muito cedo na dieta infantil, mesmo quando as \nmães dizem estar amamentando exclusivamente.': 0.45235109329223633, 'A legislação e o marketing de produtos que interferem na \namamentação um guia para o profissional de saúde.': 0.4539119005203247, 'Estocagem de alimentos complementares já preparados foi \npraticada por 24% das mães.': 0.44101494550704956}
2 – A amamentação exclusiva ainda é pouco praticada em todas 
as regiões.
---------------------------------------------------
input tempo de exercicios?
{'Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.5320515632629395, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5644347667694092, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5644347667694092, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6326790452003479, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez.\nnesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.5235025882720947, 'Se você não praticava exercício, você pode começar a fazêlo durante a gravidez. nesse caso, é melhor começar depois do 3º mês (12 semanas) de gestação.': 0.5235025882720947, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5596364736557007, 'Além da caminhada, você pode fazer treinos na esteira, ds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.5253979563713074, 'Além da caminhada, você pode fazer treinos na esteira,\nds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.5253979563713074}
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input A sobrecarga emocional e fisica esta grande
{'A gravidez é um período de sobrecarga física e emocional, e corresponde a um trabalho braçal de \nintensidade média.': 0.596531093120575, 'Tanto o estresse físico quanto o excesso de trabalho ou \natividade física extenuante, quanto o estresse psicológico estão associados com o aumento \nde casos de parto prematuro.': 0.534694492816925, 'Essa adaptação pode levar a crises emocionais, físicas e sociais.': 0.4930781126022339, 'As crises da adolescência e da \ngravidez associadas determinam conflitos emocionais mais profundos e intensos.': 0.46000605821609497, 'Todos são igualmente importantes, mas o estresse psicológico do tratamento causa mais danos à saúde mental do \ncasal e, por isso, precisam de mais atenção.': 0.4634782075881958, 'No quarto mês de gestação, as mudanças no corpo são mais evidentes, e o \nconflito com relação á autoimagem se exacerbam.': 0.4261624217033386, 'Estresse\nComo já descrito, situações de estresse levam à maior liberação de catecolaminas, que \npromovem a contração uterina.': 0.4526676535606384, 'Assim, nos dados da Região Sul e Sudeste, o desmame encontrase relacionado a uma percepção de fome do bebê, à idéia de que o leite materno sozinho não é suficiente para sustentar o bebê em crescimento, e aos problemas enfrentados pelas mães no período (dor de cabeça, nervosismo) que explicam o secar o leite do peito.': 0.45932090282440186, 'Doenças maternas\nTanto as doenças crônicas maternas, como diabetes mellitus e hipertensão arterial sistêmica, quanto as desenvolvidas na gestação com a préeclâmpsia estão envolvidas no trabalho de parto espontâneo, pela liberação de catecolaminas devido ao estresse materno e fetal, e no eletivo, sendo responsáveis por diversas complicações da mãe e do concepto.': 0.43508201837539673, 'Por outro lado, a dor, o desconforto, o estresse, a ansiedade, \no medo, a insegurança e a falta de autoconfiança podem inibir a liberação da ocitocina, \nprejudicando a saída de leite da mama.': 0.45959174633026123}
A gravidez é um período de sobrecarga física e emocional, e corresponde a um trabalho braçal de 
intensidade média.
---------------------------------------------------
input Como saber se tenho doenças?
{'b) Pesquisa de infecções\n Viroses Investigar as viroses no período pré-concepcional nos dá informações sobre a situação da doença se é imune, susceptível ou ativa.': 0.4413679838180542, 'História atual das queixas e duração\nA história atual também é chamada, equivocadamente, de história da moléstia atual, como \nse a gravidez fosse uma doença.': 0.417660117149353, 'Assim, podemos citar a) Dados pessoais nome, endereço, idade, biotipo, estado civil, naturalidade, profissão.\nb) Queixa e duração (se houver).\nc) História pregressa da moléstia atual (se houver).\nd) Antecedentes pessoais doenças como hipertensão, endocrinopatias, cardiopatias, doenças sexualmente transmissíveis e outras. vacinação, hábitos e vícios. "\ne) Antecedentes familiares hipertensão arterial sistêmica, diabetes melito, cardiopatias congênitas, malformações, gestação gemelar, doenças transmissíveis, mastopatias etc.\nf) História fisiológica desde o nascimento. menarca, ciclos menstruais, atividade sexual (frequência, início, queixas).\ng) Antecedentes obstétricos (detalhar cada gestação).\nh) Atividade profissional (fatores de risco)."': 0.4184882640838623, 'As gestantes que sabem da infecção durante o pré-natal têm indicação de \ntratamento com medicamentos para prevenir a transmissão para o feto.': 0.4220998287200928, 'Estudos têm relatado que a infecção viral é baixa e não está associada a anormalidades congênitas.': 0.42810285091400146, 'TSH\nA pesquisa de doenças da tireoide no pré-natal justificase por acometerem mulheres em \nsua fase reprodutiva.': 0.44390249252319336, 'A anorexia na criança doente é um fenômeno universal e pode \npersistir além do episódio da doença.': 0.432842493057251, 'A mãe é a \nresponsável pelo RN, exceto em situações de doença psíquica.': 0.41644036769866943, 'O manejo desta condição inclui6\n\t Identificar e tratar a causa básica que está contribuindo para a isquemia do mamilo.': 0.4285390377044678, 'Por esse motivo, a não ser que haja suplementação exógena adequada, a maioria das gestantes evolui para anemia ferropriva.': 0.4159884452819824}
TSH
A pesquisa de doenças da tireoide no pré-natal justificase por acometerem mulheres em 
sua fase reprodutiva.
---------------------------------------------------
input Posso fazer cirurgia bariatrica
{'A cirurgia bariátrica surgiu no meio médico como opção terapêutica relevante nesses casos.': 0.6333636045455933, 'Nos últimos anos, os procedimentos cirúrgicos relacionados à cirurgia bariátrica tornaramse uma \nrealidade aceita mundialmente para o tratamento da chamada obesidade mórbida.': 0.6343990564346313, 'Na mesma situação incluemse as pacientes submetidas a cirurgia bariátrica, nas quais a \nabsorção do ácido fólico estaria diminuída.': 0.60097736120224, 'No caso de mulheres submetidas a cirurgias bariátricas, a avaliação preconcepcional é obrigatória , e o ideal é que a gravidez seja planejada.': 0.6412976384162903, 'As cirurgias bariátricas podem ser máabsortivas e restritivas, como ocorre com aquela que utiliza banda gástrica por videolaparoscopia e que, durante o pré-natal, pode exigir ajustes.': 0.6468786597251892, 'deve-se levar em consideração que, na prática, nem todas as mulheres submetidas a cirurgias \nbariátricas engravidam com o peso ideal.': 0.6184722781181335, 'O tratamento cirúrgico não é indicado durante a gravidez.': 0.5905007719993591, 'O tratamento cirúrgico para correção \nda hemorroida não é indicado durante a gravidez.': 0.6170218586921692, 'Também são de interesse para o pré-natalista as cirurgias sobre as mamas (redutoras, adicionais, retirada de nódulos), indicando e antecipando possíveis dificuldades para a \nefetivação do aleitamento materno.': 0.5719142556190491, 'Somandose a essas informações, também deve ser averiguado o passado de cirurgias \nsobre o colo, como traquelotomia, conização e correção cirúrgica de insuficiência cervical.': 0.5925694704055786}
As cirurgias bariátricas podem ser máabsortivas e restritivas, como ocorre com aquela que utiliza banda gástrica por videolaparoscopia e que, durante o pré-natal, pode exigir ajustes.
---------------------------------------------------
input Qual a importnacia do pre-natal?
{'Se o pré-natal de gestações espontâneas é um período especial, o resultado das técnicas de \nreprodução humana é ainda mais valorizado.': 0.4942009449005127, 'deve-se explicar à gestante que alguns exames serão realizados logo após a primeira consulta pré-natal e outros em momentos distintos da \ngravidez.': 0.45483386516571045, 'Em razão das diferentes técnicas, a colaboração do cirurgião é muito importante, tanto na \npréconcepção quanto no pré-natal, pois a disabsorção de micronutrientes pode apresentar \naspectos distintos.': 0.46289336681365967, 'O efeito protetor do leite materno tem início logo após o \nnascimento.': 0.4761742949485779, 'A amamentação exclusiva é importante também na diminuição da \nfertilidade após o parto.': 0.48130762577056885, 'Embora toda gestação seja muito valorizada, o feto da grávida tratada pelas técnicas de \nReprodução Assistida tem geralmente um significado mais especial.': 0.4703611135482788, 'Existe uma relação direta entre duração da amamentação exclusiva \ne escolaridade materna.': 0.4664068818092346, 'Outra importante vantagem do aleitamento materno é o custo.': 0.4504305124282837, 'Os recém-nascidos humanos aparentemente são indiferentes ao sal \nou podem esboçar uma leve rejeição ao mesmo.': 0.44383561611175537, 'Ainda, para as \ncrianças de seis a onze meses, foi calculada uma dieta com leite de vaca \nconsiderando a impossibilidade da criança receber leite materno (Cardápio 2).': 0.4460790753364563}
Se o pré-natal de gestações espontâneas é um período especial, o resultado das técnicas de 
reprodução humana é ainda mais valorizado.
---------------------------------------------------
input Qual a melhor idade para a muher ter um filho?
{'A \nmortalidade em crianças que nasceram até dois anos após o nascimento de um \nirmão ou cujas mães engravidaram antes que completassem dois anos, em 39 \npaíses em desenvolvimento, é consideravelmente maior do que a encontrada \nentre as crianças que têm uma diferença de dois ou mais anos com o irmão \nmais próximo (Huttly et al., 1992).': 0.5536646246910095, 'A média de idade gestacional de nascimento nas gestações \nduplas é de 36 semanas, enquanto nas gestações triplas é de 32 semanas.': 0.5567775964736938, 'IDADE\nA idade propícia para a primeira gestação seria entre 18 e 24 anos, conforme a maioria \ndos autores."Entretanto, esse critério não deve ser considerado universal, uma vez que o \ncrescimento somático e psicológico deve ser levado em conta."\nConceituase como primigesta jovem ou precoce a gestante com 16 anos ou menos.\n"As complicações mais frequentes entre essas primigestas são anemia, abortamentos, \nprematuridade, hiperemese, problemas psicológicos (gestações indesejadas), préeclâmpsia/\neclâmpsia, baixo peso ao nascer, desproporção céfalopélvica, roturas perineais e vaginais \n(hipoplasia genital, escassa distensibilidade dos tecidos), maior mortalidade perinatal."\n"Já o conceito de primigesta idosa ou tardia não é tão bem definido. para alguns autores, é \nconsiderada a idade de 28 anos, e para outros, a idade de 30, 32 ou até de 35 anos."\n"Entre as \nprimigestas tardias ou idosas, as complicações mais frequentes são abortamento, prematuridade \n(adenomiose), préeclâmpsia, dilatação cervical lenta e hipertonia (adenomiose), período expulsivo \nprolongado (resistência de partes moles), retropulsão coccígea difícil ou ausente, e roturas \nperineais (menor distensibilidade dos tecidos)."\n"Complicações decorrentes de doenças crônicas \npreexistentes (hipertensão, diabetes, colagenoses, tireoideopatias, entre outras) são mais comuns."\nA gravidez nos extremos de idade aumenta a frequência de partos operatórios.\n"Somado à maior morbidade perinatal, existe maior prevalência de cromossomopatias \n(principalmente trissomias) nesses grupos, sobretudo nas gestantes com idade acima de 35 anos."': 0.5753396153450012, 'Existem muitas dúvidas sobre o momento ideal para iniciar o pré-natal.Alguns autores acreditam que deveria ser o mais precoce possível, enquanto outros acham que deveria ocorrer no final do primeiro trimestre.': 0.5578789114952087, 'Antigamente, denominavase pósdatismo quando a idade gestacional ultrapassava \nas 40 semanas, mas esse termo foi abandonado por não ser possível determinar a exata data \nprevista para o parto.': 0.5523194670677185, 'Restringindo a questão ao plano reprodutivo, a maioria desses jovens, em nosso meio, chega \nà maturidade sexual antes de atingir a maturidade social, emocional ou a independência \neconômica.': 0.5536219477653503, 'Após 6 meses a 1 ano do nascimento do bebê, o esperado é que você retorne\nao peso quetinha antes de engravidar.': 0.5838533639907837, 'História menstrual\nUm intervalo mínimo ideal entre a menarca e a gestação deve ser considerado quando a gravidez ocorre em adolescentes.': 0.5660567283630371, '“A Gravidez é o melhor período para\nvocê adquirir hábitos saudáveis e leválos\npara sua vida toda!”\n \nOrientações para o puerpério\nO período após o parto chamase puerpério, também conhecido como pós\nparto.': 0.5643808841705322}
Após 6 meses a 1 ano do nascimento do bebê, o esperado é que você retorne
ao peso quetinha antes de engravidar.
---------------------------------------------------
input O q nao e aconselhavel na gravidez?
{'Não \nhá indicação para interrupção da gestação, exceto em casos específicos.1,2,11\nCaxumba Contraindicada na gestação.': 0.5512920618057251, 'O uso do ácido fólico na préconcepção costuma ser recomendado para qualquer mulher que pretenda engravidar, como já foi exposto .': 0.5890262722969055, 'Raiva A gravidez não é contraindicação para o uso da profilaxia para a raiva.': 0.5686073899269104, 'Vale salientar que a orientação contraceptiva visando a evitar a gestação, que na grande maioria das vezes é indesejada, deve ser estimulada.': 0.5459018349647522, 'Inferese disso a indicação de restringir o consumo de tabaco durante a gravidez.': 0.5522551536560059, 'Sobre o uso de medicamentos, Orienta-se a gestante a não ingerilos sem prescrição médica.': 0.5591331720352173, 'Caso seja aplicada em mulheres em idade fértil, estas devem ser orientadas a evitar engravidar por quatro semanas após a imunização.': 0.5555360317230225, 'Muitas avós transmitem às suas filhas ou noras suas experiências com a amamentação, as quais, em muitos casos, são contrárias às recomendações \natuais das práticas alimentares de crianças, como, por exemplo, o uso de água, chás e outros leites nos primeiros seis meses.': 0.5530086755752563, 'Não existe \nindicação de restringir a ingestão de alimentos pelo fato da criança encontrar\nse doente.': 0.5398154854774475, 'Portanto, o \ntratamento é recomendado mesmo em mulheres assintomáticas.': 0.5699649453163147}
O uso do ácido fólico na préconcepção costuma ser recomendado para qualquer mulher que pretenda engravidar, como já foi exposto .
---------------------------------------------------
input Comer carne crua faz mal?
{'A paciente deve ser orientada a evitar carnes cruas ou mal cozidas.': 0.6557118892669678, 'Na Região Sudeste, foram considerados bons para a saúde as \ncarnes, principalmente de alguns tipos de peixe (parati, cará), frango (carne \nbranca, fígado), legumes, verduras, arroz e feijão.': 0.6237770915031433, 'No Sudeste, frutas à tarde e feijão à \nnoite fazem mal.': 0.5785785913467407, ' “pesados/fortes” – Nordeste feijão. CentroOeste carne \ngorda, feijão, fritura, ovo, peixe. Sudeste carne de porco, \ncarnes salgadas, comida gordurosa. Sul carne de porco, \ngoiaba, comida gordurosa, leguminosas, mamão, manga, \nmingau de milho, pepino, ovo.': 0.5939467549324036, 'É grande a lista de alimentos que, segundo as mães, devem ser \nevitados na alimentação das crianças pequenas pela possibilidade de causarem \nefeitos indesejáveis no funcionamento do corpo e na saúde da criança \n \n “reimosos” (que provocam inflamação) – Norte camarão, \ncaranguejo, carne de porco, jacaré, ovo, pato, peixe de couro, \npirarucu. Nordeste ata/pinha, carne de porco, peixe de água \ndoce, peixe de couro, peru. CentroOeste carne de porco. \nSudeste lingüiça.': 0.5883869528770447, 'Eles podem ser outras carnes consumidos de diversas maneiras processadas em saladas, em preparações quentes (salsicha, linguiça, (cozidos, refogados, assados, em nuggets).': 0.5785219669342041, ' “proibidos pela religião” – Norte camarão, carne de porco.': 0.710149347782135, 'Nesse grupo, já observamos o \nuso da carne bovina, de frango, de cereais e derivados e de frutas.': 0.607964813709259, 'Por “preparação” entendese os alimentos elaborados com \nmodificação na sua estrutura – como, carne cozida, frango assado, etc, ou \nalimentos reunidos como em uma salada – por exemplo. de batata com \nalface e tomate.': 0.5927004218101501, 'Evitar bacon,frios e\ntodos os dias.': 0.6376160383224487}
 “proibidos pela religião” – Norte camarão, carne de porco.
---------------------------------------------------
input o sobrepeso pode causar problemas?
{'Gestantes com sobrepeso devem \nganhar até 0,9 kg no primeiro trimestre, e gestantes obesas não necessitam ganhar peso no \nprimeiro trimestre.': 0.696658194065094, 'Mulheres muito magras ou com excesso de peso são mais sujeitas a maior incidência de problemas durante a gestação, como, por exemplo, crescimento fetal restrito e préeclâmpsia.': 0.6828881502151489, 'Baixo peso (IMC<20)\nPacientes com baixo peso tem risco aumentado para prematuridade, baixo peso ao nascer e \nanemia materna.': 0.6686152219772339, 'Ganho de peso na gravidez\nO ganho de peso na gestação está diretamente relacionado ao peso fetal ao nascer, que \nimpacta na morbimortalidade neonatal e ainda implica risco futuro de desenvolvimento de \ndiabetes, hipertensão, problemas cardíacos ou psiquiátricos na vida adulta, relacionados aos \ndesvios de peso ao nascer, para mais ou para menos.': 0.6589275598526001, 'Existem trabalhos que mostram que nas \npacientes obesas tipos 2 e 3, a perda de peso de até 4 kg na gestação pode ser tolerada ou \naté mesmo benéfica para o binômio maternofetal.': 0.6491122841835022, 'Existe \numa associação direta entre o IMC e o risco de macrossomia, o que é decorrente da resistência \nà insulina aumentada em grávidas obesas, o que leva à hiperinsulinemia fetal, importante \nfator para o crescimento intrauterino.': 0.6471880674362183, 'No entanto, cerca de 2/3 das mulheres ganham mais peso do que o recomendado, \no que leva a complicações durante a gestação, além de contribuir para a retenção de peso pós\nparto e, com isso, para o desenvolvimento da obesidade e suas complicações ao longo da vida.': 0.7202991843223572, 'Em \ncontrapartida, há indícios de que a obesidade em crianças pequenas esteja \naumentando, o que não pode ser desconsiderado, pela relação existente \nentre obesidade na infância e na fase adulta, acarretando aumento das \ndoenças crônicas nãotransmissíveis.': 0.6689111590385437, 'Fetos macrossômicos têm mais distócia de ombro \ne predisposição à obesidade na vida adulta.': 0.6842100024223328, 'A gestação está incluída na lista dos fatores clássicos desencadeantes da obesidade, e o início \nou manutenção da obesidade nessa fase está associado a inúmeros riscos maternos e fetais.': 0.6523200273513794}
No entanto, cerca de 2/3 das mulheres ganham mais peso do que o recomendado, 
o que leva a complicações durante a gestação, além de contribuir para a retenção de peso pós
parto e, com isso, para o desenvolvimento da obesidade e suas complicações ao longo da vida.
---------------------------------------------------
input sou obesa, posso ter filho?
{'Mulheres que ganham peso \ndentro dos limites propostos têm menos chance de ter filhos nos extremos de peso para a idade \ngestacional.': 0.477530837059021, 'Gestantes com sobrepeso devem \nganhar até 0,9 kg no primeiro trimestre, e gestantes obesas não necessitam ganhar peso no \nprimeiro trimestre.': 0.5458085536956787, 'Existem trabalhos que mostram que nas \npacientes obesas tipos 2 e 3, a perda de peso de até 4 kg na gestação pode ser tolerada ou \naté mesmo benéfica para o binômio maternofetal.': 0.48041200637817383, 'No caso de mulheres submetidas a cirurgias bariátricas, a avaliação preconcepcional é obrigatória , e o ideal é que a gravidez seja planejada.': 0.4724452495574951, 'deve-se levar em consideração que, na prática, nem todas as mulheres submetidas a cirurgias \nbariátricas engravidam com o peso ideal.': 0.49277955293655396, 'A empregada deve requerer a licençamaternidade até o final do primeiro mês após o parto e o benefício também se \naplica à empregada que adotar ou obter guarda judicial por fins de adoção de criança.': 0.47523677349090576, 'Um contingente significativo dos pacientes submetidos à cirurgia bariátrica é representado por mulheres, muitas delas em idade reprodutiva e com pretensão de engravidar.': 0.45841705799102783, 'Existe consenso de que a mulher que amamenta exclusivamente, em \nlivre demanda, até os seis meses após o parto e cuja menstruação ainda não \ntenha retornado, apresenta proteção contra nova gravidez (Family Health \nInternational, 1988).': 0.5208038091659546, 'Orientações para mulheres que foram submetidas à cirurgia bariátrica\n1 – Não engravidar no primeiro ano póscirurgia, porque isso compromete o período de maior perda ponderal, e a potencial perda de peso pode não ser retomada após o parto."2 – Atentarse à quantidade de alimentos ingeridos na fase de adaptação que por ser menor aumenta o risco de desnutrição."\n"3 – Evitar a gravidez por um período de 12 a 18 meses, durante a fase de perda rápida de peso, para que após a estabilização ponderal a mulher não seja mais catabólica."\n"4 – Após a perda de peso, pode aumentar o risco de engravidar, sendo necessário o uso de método anticoncepcional efetivo e apropriado."\n"6 – Durante a gravidez deve haver acompanhamento não só do obstetra, mas também do cirurgião bariátrico."\n"7 – A gestante deve receber suplemento adequado de nutrientes, em especial ácido fólico, vitamina B12, vitamina D, cálcio, ferro e tiamina na préconcepção, na gestação e no puerpério, além da suplementação nutricional adicional necessária."\n8 – recomenda-se monitoramento rigoroso do crescimento fetal durante a gestação.\n"9 – Considerar o uso de meias de compressão graduadas, hidratação e deambulação precoce no puerpério, para evitar acidentes tromboembólicos."': 0.4782490134239197, 'A mulher vacinada deve aguardar quatro semanas para iniciar nova gestação.': 0.49118268489837646}
Gestantes com sobrepeso devem 
ganhar até 0,9 kg no primeiro trimestre, e gestantes obesas não necessitam ganhar peso no 
primeiro trimestre.
---------------------------------------------------
input E sobre a vacina contra o tetano?
{'Febre tifoide Não existem dados sobre o uso da vacina anti tifoide em gestantes.': 0.8094268441200256, 'Vacinação \nA mulher deve tomar vacina contra sarampo, caxumba e rubéola (MMR), caso não seja \nimune a essas doenças, e deve ser orientada a não engravidar nos três meses seguintes.': 0.7528625130653381, 'Quando administrada em mulheres grávidas, anticorpos maternos transplacentários protegem o neonato contra a coqueluche, mas podem interferir na resposta do concepto ao ser vacinado contra DPT, diminuindo a resposta imunológica a essa vacina.': 0.7012888193130493, 'A vacina antirrubéola por vírus atenuado é secretada no leite, mas há apenas relatos de infecção assintomática.': 0.7347394227981567, 'Para a indicação da dTpa na gestante, não importa o intervalo de tempo desde a última dose de dT, posto que é essa vacina que vai proteger o recém-nascido contra a coqueluche, até que seu esquema vacinal esteja completo.': 0.7576283812522888, 'São contraindicadas durante a gestação as vacinas de vírus vivos, como é o caso da tríplice viral (sarampo, caxumba e rubéola), varicela (catapora) e papilomavírus humano (HPV), que poderão ser aplicadas no puerpério e durante a amamentação.': 0.7170673608779907, 'Rubéola\nTodas as mulheres em período reprodutivo deveriam ser rastreadas para a rubéola, e as mais \nsuscetíveis deveriam ser vacinadas antes da gestação, uma vez que não há tratamento que \nprevina a transmissão vertical.': 0.7345820665359497, 'Antipneumocócica (PCV 13 e PPV23) Até o momento, não existem evidências de que se deve indicar a vacina conjugada pneumocócica às gestantes.': 0.7937124371528625, 'Apesar de ser pouco estudada, o uso de imunoglobulina \nantivaricela (VZIG) é indicada a todas as gestantes suscetíveis e que tenham sido expostas.': 0.7073712944984436, 'Também deve-se vacinar contra a hepatite B durante a gestação se o antígeno for obtido \nda cápsula viral.': 0.763326108455658}
Febre tifoide Não existem dados sobre o uso da vacina anti tifoide em gestantes.
---------------------------------------------------
input Quais os exames que eu devo fazer?
{'deve-se explicar à gestante que alguns exames serão realizados logo após a primeira consulta pré-natal e outros em momentos distintos da \ngravidez.': 0.4540533423423767, 'Se houver necessidade, a grávida deve ser avaliada por um dentista \ne, se for preciso realizar algum procedimento, devem ser recomendados a utilização de \nanestésico local sem vasoconstritor e exames radiológicos, realizados com proteção do \navental de chumbo.': 0.4546580910682678, 'Outros exames, como eletrocardiograma, devem ser solicitados de acordo com a avaliação clínica obstétrica da gestante.': 0.46728384494781494, 'Na avaliação inicial da grávida, deverão ser discutidos seu estado geral e os exames \nlaboratoriais a serem solicitados.': 0.4703505039215088, 'Muitos exames que foram realizados no início da avaliação para o tratamento de \nreprodução assistida não terão de ser repetidos, a não ser que haja dúvidas.': 0.444122314453125, 'O exame deve ser realizado por volta de 35 semanas, \npara que se promova a profilaxia intraparto.': 0.5057990550994873, 'A citologia oncótica deve ser realizada se a \ngestante não a tiver feito recentemente.': 0.5027661323547363, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.503543496131897, 'Exame físico ginecoobstétrico\nEm termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal \ndeverá considerar o exame das mamas e o exame obstétrico propriamente dito.': 0.46578407287597656, 'A ultrassonografia é \num exame complementar importante para o diagnóstico dessa afecção, apesar de não ser \nhabitualmente realizada.': 0.4525521397590637}
O exame deve ser realizado por volta de 35 semanas, 
para que se promova a profilaxia intraparto.
---------------------------------------------------
input Qual a importancia da do diagnostivo na gravidez?
{'O médico deverá ajudar a paciente a entender a \nimportância de sua participação no processo gestacional.': 0.5353838801383972, '3.2.2 A Importância do Aleitamento Materno \n \n São inúmeras as vantagens da amamentação, especialmente nos \nprimeiros meses de vida.': 0.5921742916107178, 'Nas primeiras doze semanas de gestação, a grávida de fertilização recebe hormônios em \nquantidades importantes para que se mantenha a gravidez, e isso faz diferença, já que a \ngravidez natural normalmente dispensa essa medicação.': 0.5248714685440063, 'Emocional O papel da mãe começa desde o planejamento da gravidez, mas é durante a gestação que se acentua.': 0.5745140910148621, 'Embora toda gestação seja muito valorizada, o feto da grávida tratada pelas técnicas de \nReprodução Assistida tem geralmente um significado mais especial.': 0.5424766540527344, '6.3 Tópicos importantes para serem abordados com as mulheres na fase inicial do Aleitamento Materno\nAlém da importância do Aleitamento Materno e das desvantagens da introdução precoce de outros alimen\ntos na dieta da criança pequena, os seguintes aspectos devem ser abordados com as mu\nlheres que planejam amamentar os seus filhos\n6.3.1 Noções de fisiologia da lactação\nO leite produzido nos alvéolos é levado até os seios lactíferos por uma rede de ductos.': 0.5481758117675781, 'A amamentação exclusiva é importante também na diminuição da \nfertilidade após o parto.': 0.6319683790206909, '6.4 Importância do envolvimento da família\nA amamentação é fortemente influenciada pelo meio onde está inserida a nutriz.': 0.5430153608322144, 'Mechas, luzes e o uso de hena \nestão liberadas pelos obstetras após a primeira metade da gestação.': 0.5240753889083862, 'Existe uma relação direta entre duração da amamentação exclusiva \ne escolaridade materna.': 0.5269944667816162}
A amamentação exclusiva é importante também na diminuição da 
fertilidade após o parto.
---------------------------------------------------
input Qual a pressao arterial ideal?
{'A pressão arterial também poderá ser medida com a mulher em decúbito lateral esquerdo, no \nbraço direito, mas nunca em posição supina (deitada de costas). Existe uma correlação entre a idade gestacional e medida da altura uterina.': 0.6562415361404419, 'Considera-se hipertensão arterial na gestação níveis tensionais iguais ou maiores que 140 \nmmHg de pressão sistólica, e iguais ou maiores que 90 mmHg de pressão diastólica.': 0.7257440090179443, 'É importante frisar que uma medida de pressão arterial de 130 x 80 mmHg é aceitável no \nfinal da gravidez, mas deve chamar atenção no segundo trimestre, quando são observados \nvalores mais baixos (ao redor de 105 x 60 mmHg).': 0.7347093820571899, 'PRESSÃO ARTERIAL (PA)\nA pressão arterial (PA) é um dos mais importantes indicadores das condições de saúde da grávida, \numa vez que o diagnóstico da hipertensão se baseia na medida casual da pressão arterial.': 0.678204357624054, 'A presença de pressão arterial diastólica ≥ 110 mmHg em uma única oportunidade ou aferição.': 0.6957628726959229, 'Definese hipertensão na gravidez como pressão arterial sistólica ≥140mmHg e/ou pressão arterial diastólica ≥90mmHg, em pelo menos duas aferições, após intervalo de repouso de 15 minutos.': 0.7170179486274719, 'O aumento de 30 mmHg ou mais na pressão sistólica (máxima) e/ou de 15 mmHg ou \nmais na pressão diastólica (mínima), em relação aos níveis tensionais prégestacionais e/ou \nconhecidos até a 16a semana de gestação.': 0.6448172926902771, 'As alterações da resistência vascular periférica levam à redução dos níveis de pressão arterial, \no que já se inicia precocemente (ao redor da 6a semana), chegando a ser observada queda \nem até 15 e 5 mmHg nas pressões diastólica e sistólica no segundo trimestre.': 0.6007958054542542, 'Ao se analisar o efeito da posição nos valores da PA, deve-se considerar que, no \nfinal da gravidez, em decorrência do crescimento do útero, ocorre compressão da veia cava \ninferior e da aorta abdominal, levando à redução do retorno venoso e do débito cardíaco, \ncomprometendo, assim, os valores da PA. Os estudos que compararam a medida da pressão \narterial nas diferentes posições mostram valores significativamente mais baixos no decúbito \nlateral esquerdo.': 0.6021996736526489, 'Valores normais oscilam entre 90 e 92 cm no final da gestação.': 0.5957448482513428}
É importante frisar que uma medida de pressão arterial de 130 x 80 mmHg é aceitável no 
final da gravidez, mas deve chamar atenção no segundo trimestre, quando são observados 
valores mais baixos (ao redor de 105 x 60 mmHg).
---------------------------------------------------
input Problemas cardiacos?
{'Diabetes, hipertensão, infecção urinária, cardiopatias, tireoideopatias, apendicite, \ncolecistite, torção de tumores anexiais ou miomas pediculados são alguns problemas clínicos \ne cirúrgicos que aumentam o risco para a mãe e seu concepto.': 0.4818578362464905, 'Neste ponto é necessário ressaltar que rastrear processos patológicos latentes, entre eles a cardiopatia, as endocrinopatias e erros posturais, \né de extrema importância.': 0.5233381986618042, '\tRN com cianose ou hipoxemia persistentes (suspeita de cardiopatia \ncongênita).': 0.5042962431907654, 'Bloqueio atrioventricular fetal e sofrimento fetal em fase terminal são outras causas.': 0.5120536088943481, 'Alterações vasculares retinianas não são comuns, exceto em patologias como síndromes hipertensivas ou diabetes.': 0.4848130941390991, 'Diferentes técnicas são descritas na prática e podem conduzir a problemas disab\nsortivos, como deficiência de várias vitaminas, herniações, torções intestinais, entre outras \nintercorrências.': 0.47415441274642944, 'Entre os antecedentes familiares, os principais são diabetes, hipertensão, cardiopatias e gemelaridade, principalmente da família materna.': 0.5027119517326355, 'A agenesia renal com \nhipoplasia pulmonar também é uma das malformações que se destacam como causa de \nparto prematuro.': 0.49610352516174316, 'ANTECEDENTES PESSOAIS\nProblemas relacionados à amamentação, deambulação ou afecções do sistema osteoarticular ocorridas na infância da paciente podem causar repercussão na gestação, por influência no desenvolvimento da pelve.': 0.46436965465545654, 'Piora progressiva pode associarse a intercorrências clínicas ou \nobstétricas.': 0.4928722381591797}
Neste ponto é necessário ressaltar que rastrear processos patológicos latentes, entre eles a cardiopatia, as endocrinopatias e erros posturais, 
é de extrema importância.
---------------------------------------------------
input o que é sopro?
{'3.2 Aleitamento Materno 3.2.1 Definições Este documento adota as mesmas categorias de aleitamento materno sugeridas pela OMS, assim definidas (Organización Panamericana de la Salud/Organización Mundial de la Salud, 1991)  Aleitamento materno exclusivo – quando a criança recebe somente leite materno, diretamente da mama ou extraído, e nenhum outro líquido ou sólido, com exceção de gotas ou xaropes de vitaminas, minerais e/ou medicamentos.': 0.4673305153846741, 'O entendimento da fisiologia nos permite diferenciar os eventos normais dos patológicos.': 0.44061756134033203, 'O efeito protetor do leite materno tem início logo após o \nnascimento.': 0.451638400554657, 'Esses alimentos são \nconsiderados fortes no sentido de que têm ferro.': 0.4743776321411133, 'Com relação ao olfato, não se sabe se o feto humano responde ao \nestímulo olfatório, apesar de o sistema estar bem desenvolvido antes do \nnascimento (Bossey, 1980. Nakashima et al., 1985).': 0.4368513822555542, 'Na espécie humana, há evidências de que o líquido amniótico é aromático e que o seu odor seja influenciado pelos alimentos ingeridos pela \ngestante (Hauser et al., 1985).': 0.47334879636764526, 'A \ntécnica de sucção da mama e da mamadeira/chupeta são distintas (Neifert et \nal., 1995).': 0.45303547382354736, 'CORIONICIDADE\nÉ muito importante, quando se fala em corionicidade, saber a zigoticidade.': 0.4880183935165405, 'Poucas horas após o parto, recém-nascidos que são amamentados reconhecem as suas mães pelo cheiro (Schaal, 1988), o mesmo não \nocorrendo com as crianças não amamentadas (Cernoch e Porter, 1985).': 0.4738691449165344, 'Talvez a semelhança de aromas entre o líquido amniótico e o leite materno faça com que o recém-nascido tenha preferência pelo cheiro do \nleite materno (Stafford et al., 1976. Schaal, 1988) e pelas mamas da mãe não lavadas (Varendi et al., 1994).': 0.43985623121261597}
CORIONICIDADE
É muito importante, quando se fala em corionicidade, saber a zigoticidade.
---------------------------------------------------
input Queria saber mais sobre ovarios
{'A água e/ou sabão, por exemplo, podem ser \nescassos, e podem haver muitas demandas sobre o tempo da mãe.': 0.41361117362976074, 'O Estudo Qualitativo das Regiões Sul, Sudeste e CentroOeste \nindicaram que as avós e as vizinhas são procuradas em busca de conselhos \nquando as crianças não querem comer, constituindose em uma importante \nfonte de informações.': 0.4271812438964844, 'Em se tratando de médias, fica evidente que algumas crianças vão \nnecessitar de mais ou menos energia provenientes dos alimentos \ncomplementares.': 0.4114145040512085, 'Nos casos de gestação repetida, questionar também sobre a evolução do puerpério, \ntentando identificar informações sobre depressão puerperal, infecções e a experiência com \na amamentação.': 0.4940102696418762, 'Saber sobre as parcerias sexuais pode ser um indicativo da necessidade de aprofundar os questionamentos sobre doenças transmitidas por meio do ato sexual e os tratamentos realizados.': 0.47235608100891113, '6.3 Tópicos importantes para serem abordados com as mulheres na fase inicial do Aleitamento Materno\nAlém da importância do Aleitamento Materno e das desvantagens da introdução precoce de outros alimen\ntos na dieta da criança pequena, os seguintes aspectos devem ser abordados com as mu\nlheres que planejam amamentar os seus filhos\n6.3.1 Noções de fisiologia da lactação\nO leite produzido nos alvéolos é levado até os seios lactíferos por uma rede de ductos.': 0.4276012182235718, 'Fazem parte do manejo da mastite4\nManutenção da amamentação.': 0.41429638862609863, 'A \ntécnica de sucção da mama e da mamadeira/chupeta são distintas (Neifert et \nal., 1995).': 0.4163234829902649, 'Entre o nascimento e a última procedência, pode haver uma gama de informações relevantes, principalmente no que se refere às infecções e infestações.Ainda nesse item deve-se questionar o eventual tipo de assistência pré-natal recebida até o momento e as intervenções que já foram iniciadas.': 0.443835973739624, 'Há necessidade de complementar esse item com a história de tratamento do parceiro sexual.': 0.47383058071136475}
Nos casos de gestação repetida, questionar também sobre a evolução do puerpério, 
tentando identificar informações sobre depressão puerperal, infecções e a experiência com 
a amamentação.
---------------------------------------------------
input A violencia pode prejudicar minha gravidez
{'Como abordagem inicial, vale ressaltar que a gestação poderá prejudicar a adolescente se ela \nainda se encontrar em fase de desenvolvimento.Além disso, em alguns casos, pode dificultar o parto transvaginal por causa do pouco desenvolvimento da bacia e das partes moles, embora os maiores problemas enfrentados pelas adolescentes com a gestação sejam os de natureza psicossocial, o que determina, muitas vezes, a não realização da assistência pré-natal e a ocultação da gravidez. Podem ocorrer muitos outros problemas, como anemia, uso de drogas lícitas e ilícitas, crescimento fetal restrito e préeclâmpsia.\n': 0.6788949966430664, 'Muitos medicamentos e fitoterápicos podem ter efeitos danosos para o feto em qualquer momento da gravidez, causando malformações congênitas (teratogênese), prejuízo para o crescimento e o desenvolvimento funcional do feto, ou efeitos tóxicos sobre os tecidos fetais.': 0.7223256826400757, 'Hábitos como o uso de álcool, drogas e o fumo, assim como doenças sexualmente \ntransmissíveis, anemia, desnutrição, abuso sexual e físico, podem, com frequência, aumentar \na morbidade para a mãe e para o feto.': 0.666416585445404, 'Algumas infecções apresentam claro potencial de transmissão vertical e, nesses casos, a \ngestante deve ser vista como potencial vítima dessa forma de disseminação da infecção.': 0.6488803029060364, ' deve-se evitar o uso da mamadeira, pois a mesma pode \natrapalhar a amamentação e é importante fonte de \ncontaminação e transmissão de doenças.': 0.6728981137275696, 'Essa prática pode ser prejudicial, uma vez que a \nmamadeira é uma importante fonte de contaminação, além de reduzir o \ntempo de sucção das mamas, interferindo na amamentação sob livre \ndemanda, alterar a dinâmica oral e retardar o estabelecimento da lactação \n(Hollen, 1976. Monte et al., 1997. World Health Organization 1998b).': 0.6720751523971558, 'As doenças infecciosas podem ter importantes repercussões sobre o feto, podendo ser \nresponsáveis por abortamentos, prematuridade e óbito fetal.': 0.6691393852233887, 'Entretanto, não existe indicação formal para interrupção da gestação, pois os \nvírus utilizados na imunização são teoricamente menos agressivos que as cepas selvagens, \nsendo menor o risco de dano ao feto.': 0.6620122194290161, 'A história sexual e o uso de métodos anticoncepcionais devem ser explorados, devido principalmente ao risco de doença sexualmente transmissível (DST).': 0.6727150082588196, 'A descendência direta de alguns grupos raciais pode significar risco adicional de algumas doenças.': 0.6728175282478333}
Muitos medicamentos e fitoterápicos podem ter efeitos danosos para o feto em qualquer momento da gravidez, causando malformações congênitas (teratogênese), prejuízo para o crescimento e o desenvolvimento funcional do feto, ou efeitos tóxicos sobre os tecidos fetais.
---------------------------------------------------
input Tive problemas ginecologicos, posso engravidar?
{'Sim, desde que você esteja realizando o pré-natal e não apresente nenhuma\ncomplicação na gravidez.': 0.5216395258903503, 'A tudo isso podemos somar a eventualidade de a gravidez ocorrer em um \norganismo com algum tipo de doença, caracterizando as intercorrências clínicocirúrgicas.': 0.5184571743011475, 'Caso engravide descompensada, a paciente merecerá reavaliação quanto à necessidade de prevenir o abortamento por insuficiência lútea, o que poderá \nocorrer e não é raro em mulheres com doenças da tireoide.': 0.6186302900314331, 'História de cirurgias ginecológicas, principalmente no colo uterino e na mama, deve ser investigada.': 0.5571614503860474, 'Antecedentes ginecológicos\nNa primeira consulta de pré-natal, é necessário averiguar o passado ginecológico da \ngestante, que dará informações importantes sobre a possibilidade de caracterizar eventuais \nriscos ao processo gestacional em curso, tais como cirurgias prévias no útero, insuficiência \nlútea, perdas gestacionais prévias, insuficiência cervical, trabalho de parto prétermo e pré\neclâmpsia, entre outras.': 0.5704128742218018, 'Até o terceiro mês, pode acontecer um sangramento devido ao descolamento do saco \ngestacional, que está se moldando ao útero.': 0.5499429106712341, 'Existe consenso de que a mulher que amamenta exclusivamente, em \nlivre demanda, até os seis meses após o parto e cuja menstruação ainda não \ntenha retornado, apresenta proteção contra nova gravidez (Family Health \nInternational, 1988).': 0.5255691409111023, 'À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.': 0.650774359703064, 'Dessa forma, a prática de atividade sexual pode ser interrompida em casos de ameaça de aborto, \nsangramento vaginal, ameaça de parto prematuro, placenta prévia ou insuficiência cervical.': 0.5685366988182068, 'Aqui deve ser lembrada a retirada de leiomioma, a cesárea e eventual rafia para correção de ruptura \nuterina.': 0.535700798034668}
À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.
---------------------------------------------------
input Tive problemas  obstetricos, posso engravidar?
{'Sim, desde que você esteja realizando o pré-natal e não apresente nenhuma\ncomplicação na gravidez.': 0.5590651631355286, 'Caso engravide descompensada, a paciente merecerá reavaliação quanto à necessidade de prevenir o abortamento por insuficiência lútea, o que poderá \nocorrer e não é raro em mulheres com doenças da tireoide.': 0.6561073660850525, 'A tudo isso podemos somar a eventualidade de a gravidez ocorrer em um \norganismo com algum tipo de doença, caracterizando as intercorrências clínicocirúrgicas.': 0.547707736492157, 'Até o terceiro mês, pode acontecer um sangramento devido ao descolamento do saco \ngestacional, que está se moldando ao útero.': 0.5805635452270508, 'Amputações do colo uterino para tratamento de neoplasias cervicais, seja pela cirurgia \nconvencional, seja através de cirurgias de alta frequência (CAF), também podem levar \nao encurtamento do colo e à insuficiência cervical em caso de futura gravidez.Como consequência, poderão ocorrer abortamentos tardios e partos prematuros.': 0.5444853901863098, 'À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.': 0.661014199256897, 'Antecedentes obstétricos\nEntre as informações importantes sobre o passado obstétrico, elencamse a data da última menstruação (DUM), número de gestações e intervalo entre elas, evolução no pré-natal (aborto, \ndiabetes, parto prétermo, restrição de crescimento intraútero ou baixo peso ao nascer, préeclâmpsia/eclâmpsia, anomalias congênitas no recém-nascido, óbito fetal ou neonatal), via do \nparto e, se cesárea, qual foi a indicação e há quanto tempo foi realizada, tempo do trabalho \nde parto, condições do recém-nascido e evolução neonatal."Devem-se averiguar aspectos \npsicológicos e sociais, tais como gravidez programada ou não, medos e tabus."': 0.5595921277999878, 'Antecedentes ginecológicos\nNa primeira consulta de pré-natal, é necessário averiguar o passado ginecológico da \ngestante, que dará informações importantes sobre a possibilidade de caracterizar eventuais \nriscos ao processo gestacional em curso, tais como cirurgias prévias no útero, insuficiência \nlútea, perdas gestacionais prévias, insuficiência cervical, trabalho de parto prétermo e pré\neclâmpsia, entre outras.': 0.6016905903816223, 'Dessa forma, a prática de atividade sexual pode ser interrompida em casos de ameaça de aborto, \nsangramento vaginal, ameaça de parto prematuro, placenta prévia ou insuficiência cervical.': 0.6248428821563721, 'Aqui deve ser lembrada a retirada de leiomioma, a cesárea e eventual rafia para correção de ruptura \nuterina.': 0.5784615874290466}
À expressão, algumas pacientes já podem apresentar colostro no segundo mês de gestação.
---------------------------------------------------
input Problemas familiares podem me prejudicar?
{'ANTECEDENTES PESSOAIS\nProblemas relacionados à amamentação, deambulação ou afecções do sistema osteoarticular ocorridas na infância da paciente podem causar repercussão na gestação, por influência no desenvolvimento da pelve.': 0.5417706370353699, 'Como abordagem inicial, vale ressaltar que a gestação poderá prejudicar a adolescente se ela \nainda se encontrar em fase de desenvolvimento.Além disso, em alguns casos, pode dificultar o parto transvaginal por causa do pouco desenvolvimento da bacia e das partes moles, embora os maiores problemas enfrentados pelas adolescentes com a gestação sejam os de natureza psicossocial, o que determina, muitas vezes, a não realização da assistência pré-natal e a ocultação da gravidez. Podem ocorrer muitos outros problemas, como anemia, uso de drogas lícitas e ilícitas, crescimento fetal restrito e préeclâmpsia.\n': 0.5325068235397339, 'A descendência direta de alguns grupos raciais pode significar risco adicional de algumas doenças.': 0.4900238513946533, 'ESTADO CIVIL\nAs gestantes sem parceiro fixo têm mais problemas emocionais, sexuais e socioeconômicos.': 0.4784104824066162, 'A participação das avós é bastante marcante na cultura brasileira. elas costumam exercer grande influência sobre as mães, em especial as adolescentes, o que pode favorecer ou dificultar a amamentação.': 0.5097817778587341, 'Intercorrências clínicocirúrgicas\nA ocorrência de algumas doenças durante a gestação pode interferir no prognóstico materno\nfetal.': 0.5465622544288635, 'As doenças infecciosas podem ter importantes repercussões sobre o feto, podendo ser \nresponsáveis por abortamentos, prematuridade e óbito fetal.': 0.45683908462524414, 'Alterações posturais podem afetar temporariamente o débito, por interferirem no retorno venoso.': 0.5287319421768188, 'Posteriormente, esse autocontrole sofrerá influência de outros \nfatores.O mais importante deles é o fator cultural relacionado à alimentação.': 0.45365744829177856, 'Essa adaptação pode levar a crises emocionais, físicas e sociais.': 0.5236310958862305}
Intercorrências clínicocirúrgicas
A ocorrência de algumas doenças durante a gestação pode interferir no prognóstico materno
fetal.
---------------------------------------------------
input As mamas se modificam na gravidez?
{'Modificações mamárias\nAs mamas sofrem profundas transformações durante o ciclo gravídicopuerperal, e, ao contrário dos outros órgãos, não volta ao estado prégravídico ao final do puerpério, pois é somente nessa época da vida da mulher que esse órgão atinge seu completo desenvolvimento.': 0.6474443078041077, 'É importante frisar que a maioria das alterações da pele durante a gravidez \nconsiste em modificações funcionais decorrentes do processo gestacional.': 0.6602875590324402, 'Todos os órgãos se recuperam das alterações ocorridas ao longo da\ngravidez.': 0.6138599514961243, ' Adrenais\nAs adrenais não sofrem mudanças morfológicas durante a gestação.': 0.6623605489730835, 'Modificações musculoesqueléticas\nAs modificações musculoesqueléticas são decorrentes da embebição gravídica, alterações \nhormonais e das alterações posturais desencadeadas pelo aumento uterino.': 0.6284511089324951, 'Depois que a gestante compreender a importância da amamentação, ficará mais fácil aceitar as medidas de preparo das mamas para o aleitamento, evitando que ocorram fissuras mamilares e haja descontinuidade da amamentação.': 0.6432227492332458, 'Após o puerpério, o útero reduz suas dimensões, porém \nmantendo um volume um pouco maior do que antes da gestação.': 0.6501293778419495, 'A primeira adaptação circulatória observada na \ngestante é referente às alterações ligadas ao volume e à constituição do sangue.': 0.6159669756889343, 'Modificações cardíacas\nDurante o curso da gravidez, o coração sofre alterações de posição, volume e função.': 0.6222299337387085}
 Adrenais
As adrenais não sofrem mudanças morfológicas durante a gestação.
---------------------------------------------------
input Exames vaginais 
{'O toque vaginal é realizado após o exame físico obstétrico, como o último item propedêutico \ndo exame físico na primeira consulta do pré-natal, cujos detalhes serão discutidos mais adiante\nExame físico obstétrico\nFaz parte do exame físico obstétrico o exame do abdome, que inclui sua inspeção, palpação, \nmensuração da altura uterina e ausculta dos batimentos cardíacos fetais (BCF).': 0.624549925327301, 'O exame abdominal deve ser minucioso, incluindo inspeção e palpação do abdome, mensuração da altura \nuterina e ausculta dos BCF.': 0.6438259482383728, 'Complementase o exame físico obstétrico com o toque vaginal (na maioria das vezes, \nbidigital), combinado com a palpação uterina.': 0.7763680219650269, 'A palpação das glândulas mamárias e do segmento axilar e da axila é um ponto fundamental \nnesse exame.': 0.6277621388435364, 'Para deixar a realização do exame físico mais cômoda para a paciente, o exame do ânus para verificar a presença de hemorroidas pode ser realizado durante o exame físico ginecoobstétrico.': 0.6172676682472229, 'No início da gravidez ele permite detectar \neventual encurtamento cervical e dilatação não detectada no exame com espéculo vaginal.': 0.6292365789413452, 'Exame físico ginecoobstétrico\nEm termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal \ndeverá considerar o exame das mamas e o exame obstétrico propriamente dito.': 0.6125186085700989, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.6313585638999939, 'Considerando os parâmetros obstétricos, a palpação abdominal busca a identificação de \neventuais massas ovarianas e avaliação do útero.': 0.6330718994140625, 'A avaliação do colo uterino por ultrassonografia (sempre transvaginal) \ncostuma ser realizada entre a 20a e a 24a semana de gestação, com a ultrassonografia \nmorfológicofetal de segundo trimestre.': 0.6276339888572693}
Complementase o exame físico obstétrico com o toque vaginal (na maioria das vezes, 
bidigital), combinado com a palpação uterina.
---------------------------------------------------
input Posso usar filtro solar?
{'A maioria dos protetores solares utiliza filtro \nquímico (fator de proteção solar, FPS), sendo recomendado o creme FPS 30 a 35.': 0.6142008304595947, 'O filtro solar deve ser reaplicado pelo menos três vezes ao dia, \nmesmo sem exposição ao sol.': 0.6556228995323181, 'Atualmente, \nexistem bloqueadores físicos com titânio para bloquear a ação da radiação ultravioleta, cujo \nuso está liberado na gestação.': 0.5320595502853394, 'Cuidados para evitar ou reduzir o risco de cloasma\nPara evitar o cloasma, Orienta-se evitar a exposição solar entre 10 e 16 horas, e usar cremes \nfotoprotetores, chapéu, sombrinha ou guardasol.': 0.6002297401428223, '\tRestrição ao uso de produtos que retiram a proteção natural do mamilo, como sabões, \nálcool ou qualquer produto secativo.': 0.514049768447876, 'Alguns cosméticos, como componentes de alguns cremes, xampus, tintura para cabelos, têm \npotencial teratogênico e devem ser evitados.': 0.5176066160202026, 'Assim, a prevenção deve ser feita com hidratação frequente \nda pele com qualquer produto, independentemente de sua formulação, desde que não seja \ncontraindicado na gravidez.': 0.5763740539550781, '\tCuidados para que os mamilos se mantenham secos, expondoos ao ar livre ou à luz solar, \ne trocas frequentes dos forros utilizados quando há vazamento de leite.': 0.5359584093093872, 'Tintura para cabelo, depilação e drenagem linfática\nApós o primeiro trimestre, estão liberadas colorações que não contenham chumbo ou \namônia.': 0.5184885263442993, 'Para higienização das mãos, o álcool é utilizado em solução com emoliente, para evitar o \nressecamento excessivo da pele.': 0.49770498275756836}
O filtro solar deve ser reaplicado pelo menos três vezes ao dia, 
mesmo sem exposição ao sol.
---------------------------------------------------
input Varizes na gravidez
{'É importante frisar que a maioria das alterações da pele durante a gravidez \nconsiste em modificações funcionais decorrentes do processo gestacional.': 0.5844129920005798, 'Alguns fatores que influenciam a gestação podem ser isolados, aditivos ou potencializados.': 0.5401771664619446, 'NACIONALIDADE\nCaracterísticas étnicas podem repercutir na gravidez, já que podem interferir na estrutura \ncorporal da gestante, como sobre seu estado nutricional e formato da pelve.': 0.5368651151657104, 'O evoluir da gestação mimetiza manifestações clínicas de doenças, na \ndecorrência das adaptações gravídicas do organismo materno.': 0.6332604885101318, 'Visando \ndar continuidade ao tema, procuramos referir esses fatores agrupandoos em relação ao \nevolver da gestação, do parto e das condições do recém-nascido.': 0.5421872735023499, 'A primeira adaptação circulatória observada na \ngestante é referente às alterações ligadas ao volume e à constituição do sangue.': 0.5378546714782715, 'A cor do leite varia ao longo de uma mamada e também com a dieta da mãe.': 0.5844429135322571, 'A falta de mudança dos BCF nas situações descritas pode ocorrer por se tratar do momento \nde sono fetal.': 0.5784195065498352, 'Esse \ntempo de duplicação tem sido adotado para diferenciar a gravidez normal da anormal, como a \ngravidez ectópica.': 0.5597051382064819, 'Essa cifra varia de acordo com a população (assim como varia a duração da \namamentação exclusiva), porém, o início precoce dos alimentos \ncomplementares é um hábito generalizado no País.': 0.5350014567375183}
O evoluir da gestação mimetiza manifestações clínicas de doenças, na 
decorrência das adaptações gravídicas do organismo materno.
---------------------------------------------------
input Tenho que preparar as mamas para amamentar?
{'Depois que a gestante compreender a importância da amamentação, ficará mais fácil aceitar as medidas de preparo das mamas para o aleitamento, evitando que ocorram fissuras mamilares e haja descontinuidade da amamentação.': 0.6402642130851746, 'Deixar o bebê mamar \naté esvaziar bem a primeira mama para então trocar de mama.': 0.5949828624725342, '\tSe o bebê não sugar, a mama deve ser ordenhada manualmente ou com bomba de extra\nção de leite.': 0.523202121257782, '\tOrdenha da mama se não houver esvaziamento adequado pelo bebê.': 0.522639811038971, 'Para a mãe que tem mamilos planos ou invertidos conseguir amamentar com sucesso, é fundamental que receba ajuda logo após o nascimento do bebê.': 0.5882596373558044, 'O mais importante é que a mãe dê tempo suficiente à criança para que esvazie adequadamente a mama.': 0.5486048460006714, 'É imperativo o diagnóstico de mamilos invertidos ou pouco desenvolvidos, e desde esse \nmomento Pode-se orientar a paciente a exercitar a expressão para facilitar o momento da \namamentação.': 0.5223256945610046, '\tAplicação do próprio leite materno ordenhado nos mamilos machucados.': 0.5759419202804565, 'Fazem parte do manejo dos traumas mamilares6\n\t Início da mamada pela mama menos afetada.': 0.5743982195854187, 'O exercício que a criança faz para retirar o leite da mama da mãe é muito importante para o desenvolvimento adequado de \nsua cavidade oral.': 0.526463508605957}
Depois que a gestante compreender a importância da amamentação, ficará mais fácil aceitar as medidas de preparo das mamas para o aleitamento, evitando que ocorram fissuras mamilares e haja descontinuidade da amamentação.
---------------------------------------------------
input Viagens estao permitidas?
{'Além disso, são oferecidos a crianças sem o devido reaquecimento.': 0.49821680784225464, 'Viagens\nAs viagens terrestres estão permitidas (incluindo a direção de veículos), desde que com a \nmovimentação frequente dos membros inferiores para evitar a formação de trombos.deve-se \nincentivar a gestante a usar sempre o cinto de segurança de quatro pontos.': 0.5507668256759644, 'Não há restrição \nde viagens aéreas até a 34a semana de gravidez.': 0.6427521705627441, 'VIAGENS \nAs viagens aéreas, geralmente, são seguras para a grávida até quatro semanas antes da data \nprovável do parto.': 0.5933384299278259, 'Vacinação no puerpério\nVacinas que utilizam vírus vivos ou inativados não interferem na segurança da amamentação.Sendo assim, nesse período, nenhuma vacina é contraindicada.': 0.49822866916656494, 'Orientações sobre a atividade sexual\nA atividade sexual está liberada durante a gravidez, e sua restrição só será recomendada em \ncaso de desconforto, sangramento gestacional ou trabalho de parto prétermo.': 0.5246114730834961, '6.3.3 Amamentação em livre demanda\nrecomenda-se que a criança seja amamentada sem restrições de horários e de duração das \nmamadas.': 0.5382125973701477, 'Também são contraindicados os esportes \ncom risco de queda, como o hipismo e o esqui.': 0.5139317512512207, '\tMamadas frequentes, sem horários préestabelecidos (livre demanda).': 0.5195367932319641, 'Mechas, luzes e o uso de hena \nestão liberadas pelos obstetras após a primeira metade da gestação.': 0.5074183344841003}
Não há restrição 
de viagens aéreas até a 34a semana de gravidez.
---------------------------------------------------
input Atividades fisicas sao importantes?
{'As atividades físicas mais adequadas são as \ncaminhadas, a natação e a hidroginástica.': 0.5247025489807129, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5584548115730286, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.5584548115730286, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.5534759759902954, 'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.5971196293830872, 'Os exercícios mais \naconselhados são caminhada, natação, hidroginástica, ioga e alongamento.': 0.5021051168441772, 'Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.48655229806900024, 'Além da caminhada, você pode fazer treinos na esteira, ds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.4808693528175354, 'Além da caminhada, você pode fazer treinos na esteira,\nds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.4808693528175354}
Exercício Físico na Gestação
 
Dúvidas frequentes 
Quantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.
---------------------------------------------------
input Posso fazer sexo na gravidez?
{'Orientações sobre a atividade sexual\nA atividade sexual está liberada durante a gravidez, e sua restrição só será recomendada em \ncaso de desconforto, sangramento gestacional ou trabalho de parto prétermo.': 0.5671902894973755, 'A revolução sexual da década de 1960 questionou a sexualidade da mulher \nresultando na liberdade de iniciar a atividade sexual sem vínculo obrigatório de casamento.': 0.5798119902610779, 'Dentre múltiplas determinações, a erotização do adolescente promovida \npela mídia estimula a iniciação sexual precoce, que, na ausência do domínio das práticas \ncontraceptivas, pode resultar em gravidez não desejada.': 0.5572149157524109, 'Durante a gravidez, grande parte das atividades físicas é segura.': 0.5552031993865967, 'Até a 12a semana de gestação é utilizada a técnica transvaginal, e, a partir dessa idade \ngestacional, a técnica transabdominal.': 0.5669846534729004, 'Existe consenso de que a mulher que amamenta exclusivamente, em \nlivre demanda, até os seis meses após o parto e cuja menstruação ainda não \ntenha retornado, apresenta proteção contra nova gravidez (Family Health \nInternational, 1988).': 0.5588710904121399, 'A amamentação exclusiva é importante também na diminuição da \nfertilidade após o parto.': 0.5626571774482727, '\tO uso de drogas para supressão da lactação não está indicado quando as mães desejam \ncontinuar a amamentação.': 0.5526174306869507, 'Mechas, luzes e o uso de hena \nestão liberadas pelos obstetras após a primeira metade da gestação.': 0.5499242544174194, 'Se o bebê continuar a recusar uma das \nmamas, é possível manter a amamentação exclusiva utilizando apenas uma delas.': 0.5548450350761414}
A revolução sexual da década de 1960 questionou a sexualidade da mulher 
resultando na liberdade de iniciar a atividade sexual sem vínculo obrigatório de casamento.
---------------------------------------------------
input Exames rotineiros sao importantens na gravidez?
{'deve-se explicar à gestante que alguns exames serão realizados logo após a primeira consulta pré-natal e outros em momentos distintos da \ngravidez.': 0.48154985904693604, 'O diagnóstico da gravidez, na atualidade, é realizado de forma rápida e simples e na maioria das vezes desconsiderando o exame físico e a anamnese. no entanto, devido à importância de o diagnóstico ser comprovado por um médico, inicialmente apresentaremos como fazer o diagnóstico clínico de gravidez, que é fundamental para os médicos obstetras e não obstetras que realizam o atendimento inicial de mulheres nos programas de saúde da família e nos serviços de emergência.': 0.46201252937316895, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.5643537640571594, 'A citologia oncótica deve ser realizada se a \ngestante não a tiver feito recentemente.': 0.468741238117218, 'Exame físico ginecoobstétrico\nEm termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal \ndeverá considerar o exame das mamas e o exame obstétrico propriamente dito.': 0.4936109781265259, 'A ultrassonografia é \num exame complementar importante para o diagnóstico dessa afecção, apesar de não ser \nhabitualmente realizada.': 0.4925570487976074, 'É nessa abordagem inicial que são solicitados os exames complementares, que, aliados à avaliação clínica, \nvão definir o estado atual da paciente."Muito embora essa avaliação seja feita de forma \nvariada em diferentes serviços, o que será amplamente discutido no capítulo 8 sugerimos \nseparar os exames complementares em quatro grupos\n Grupo 1 exames de cunho específico para o pré-natal, com interesse maternofetal \n(hemograma, tipagem sanguínea, fator Rh, urina I, urocultura, sorologias, glicemia e \nultrassonografia obstétrica)." " Grupo 2 exames determinados por intercorrências ou doenças associadas que justi\nfiquem seguimento durante a gestação (função renal, função cardíaca, fundo de olho, \ninvestigação de anemias)."" Grupo 3 exames para rastreamento de doenças potenciais na gestação, sempre \nligados às características dos serviços e do cliente, bem como as doenças prevalentes em cada região (rastreamento de diabetes, rastreamento de préeclampsia)." " Grupo 4 exames para rastreamento de doenças indiretamente ligadas à gestação. têm \nsua realização justificada pela oportunidade de estar diante da mulher que nunca teve a \noportunidade de buscar serviço médico (rastreamento de verminoses, rastreamento de \ncâncer cervical)."': 0.4513469934463501, 'Ecocardiografia fetal\nA ecocardiografia fetal é um exame anatômico e funcional do coração fetal."Pode ser \nrealizado a partir de 18 semanas de gestação, mas o período ideal é o da 28a semana."': 0.45160937309265137, 'Exames ultrassonográficos\nUltrassonografia morfológica de primeiro trimestre com perfil bioquímico\nDeve ser realizada entre 11a e 14a semanas, de preferência na 12a semana, com o objetivo de \ndetectar a síndrome de Down (89%) e a trissomia do 18 (95%).': 0.4517318606376648, 'Por esses motivos, deve ser precedido por cuidadoso inventário anmnéstico.': 0.4575616121292114}
Exame físico ginecológico deve-se seguir o padrão habitual de todo exame 
ginecológico.
---------------------------------------------------
input o que e o cartao prenatal?
{'Administrativamente, as consultas prénatais se dividem em primeira consulta (também \nchamada de caso novo de pré-natal) e os retornos.': 0.4522767663002014, 'Parametrização para a distribuição dos atendimentos durante gestação e puerpério.': 0.4417049288749695, 'Com intuito de atrair as gestantes o mais precocemente possível, o Ministério da Saúde tem estimulado as instituições com benefícios extras quando a grávida inicia \nrapidamente o pré-natal e garantese o mínimo de consultas, exames, vacinação e retorno no puerpério.': 0.4044145941734314, 'Caso já tenha sido vacinada nos \núltimos dez anos, a mulher deverá receber apenas o reforço.': 0.4446448087692261, 'Portanto, a assistência pré-natal deve ser iniciada antes da concepção, \nde forma a garantir que a mulher esteja fisicamente apta a suportar essa sobrecarga.': 0.4491841793060303, 'Existe uma relação direta entre duração da amamentação exclusiva \ne escolaridade materna.': 0.41820836067199707, 'Esse \ntempo de duplicação tem sido adotado para diferenciar a gravidez normal da anormal, como a \ngravidez ectópica.': 0.43845051527023315, 'Meningocócica conjugada Uma dose, mesmo para aquelas vacinadas na infância ou há mais de cinco anos.': 0.4062807559967041, 'É comum a criança aceitar novos \nalimentos apenas após algumas tentativas e não nas primeiras.': 0.40676969289779663, '2 – A amamentação exclusiva ainda é pouco praticada em todas \nas regiões.': 0.40835821628570557}
Administrativamente, as consultas prénatais se dividem em primeira consulta (também 
chamada de caso novo de pré-natal) e os retornos.
---------------------------------------------------
input Exercicio fisico 
{'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6800593137741089, 'EXERCÍCIOS FÍSICOS\nA prática de exercícios aeróbicos regulares durante a gravidez parece melhorar ou manter a \ncapacidade física e a boa imagem corporal da gestante.': 0.658598780632019, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.7322145104408264, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.7322145104408264, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.7282962799072266, 'As atividades físicas mais adequadas são as \ncaminhadas, a natação e a hidroginástica.': 0.6621325612068176, 'Os exercícios mais \naconselhados são caminhada, natação, hidroginástica, ioga e alongamento.': 0.6570004820823669, 'Além da caminhada, você pode fazer treinos na esteira, ds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.6527761816978455, 'Além da caminhada, você pode fazer treinos na esteira,\nds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.6527761816978455, 'Seu tratamento consiste na orientação de repouso, com os membros superiores elevados e \nutilização de meias elásticas.': 0.6083571314811707}
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Exercicio fisico especial
{'Exercício Físico na Gestação\n \nDúvidas frequentes \nQuantas vezes devo fazer exercício? Você deve se exercitar de 3 a 5 vezes na semana.': 0.6277977228164673, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação,\nbicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.7123584151268005, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates\nQue tipo de exercício NÃO posso fazer?': 0.7123584151268005, 'EXERCÍCIOS FÍSICOS\nA prática de exercícios aeróbicos regulares durante a gravidez parece melhorar ou manter a \ncapacidade física e a boa imagem corporal da gestante.': 0.6348978281021118, 'Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica\nlocalizada, Yoga ou Pilates. Quetipo de exercício NÃO posso fazer?': 0.7059603929519653, 'Os exercícios mais \naconselhados são caminhada, natação, hidroginástica, ioga e alongamento.': 0.649476170539856, 'Além da caminhada, você pode fazer treinos na esteira, ds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.6717769503593445, 'Além da caminhada, você pode fazer treinos na esteira,\nds bicicleta estacionária, hidroginástica, natação, entre outros.': 0.6717769503593445, 'As atividades físicas mais adequadas são as \ncaminhadas, a natação e a hidroginástica.': 0.6507022380828857, 'Seu tratamento consiste na orientação de repouso, com os membros superiores elevados e \nutilização de meias elásticas.': 0.6181409358978271}
Você deve realizar tanto exercícios aeróbicos como caminhada, hidroginástica, natação, bicicleta estacionária ou dança, quanto exercícios de alongamento muscular e ginástica localizada, Yoga ou Pilates
Que tipo de exercício NÃO posso fazer?
---------------------------------------------------
input Exame gineco obsetrico
{'Exame físico ginecoobstétrico\nEm termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal \ndeverá considerar o exame das mamas e o exame obstétrico propriamente dito.': 0.779857337474823, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.7482188940048218, 'Complementase o exame físico obstétrico com o toque vaginal (na maioria das vezes, \nbidigital), combinado com a palpação uterina.': 0.6305186748504639, 'O toque vaginal é realizado após o exame físico obstétrico, como o último item propedêutico \ndo exame físico na primeira consulta do pré-natal, cujos detalhes serão discutidos mais adiante\nExame físico obstétrico\nFaz parte do exame físico obstétrico o exame do abdome, que inclui sua inspeção, palpação, \nmensuração da altura uterina e ausculta dos batimentos cardíacos fetais (BCF).': 0.6379501223564148, 'Para deixar a realização do exame físico mais cômoda para a paciente, o exame do ânus para verificar a presença de hemorroidas pode ser realizado durante o exame físico ginecoobstétrico.': 0.628415048122406, 'O exame físico da gestante deve incluir aferição de peso, medida de pressão arterial, exame físico geral, exame de mamas, exame obstétrico e exame genital.': 0.6522859930992126, 'O diagnóstico da gravidez, na atualidade, é realizado de forma rápida e simples e na maioria das vezes desconsiderando o exame físico e a anamnese. no entanto, devido à importância de o diagnóstico ser comprovado por um médico, inicialmente apresentaremos como fazer o diagnóstico clínico de gravidez, que é fundamental para os médicos obstetras e não obstetras que realizam o atendimento inicial de mulheres nos programas de saúde da família e nos serviços de emergência.': 0.6013315320014954, 'O tipo de exame a ser solicitado dependerá de ser realizado na urina ou no sangue, assim como do tempo de atraso menstrual.': 0.622894823551178, 'Outros exames, como eletrocardiograma, devem ser solicitados de acordo com a avaliação clínica obstétrica da gestante.': 0.5991507768630981, 'É nessa abordagem inicial que são solicitados os exames complementares, que, aliados à avaliação clínica, \nvão definir o estado atual da paciente."Muito embora essa avaliação seja feita de forma \nvariada em diferentes serviços, o que será amplamente discutido no capítulo 8 sugerimos \nseparar os exames complementares em quatro grupos\n Grupo 1 exames de cunho específico para o pré-natal, com interesse maternofetal \n(hemograma, tipagem sanguínea, fator Rh, urina I, urocultura, sorologias, glicemia e \nultrassonografia obstétrica)." " Grupo 2 exames determinados por intercorrências ou doenças associadas que justi\nfiquem seguimento durante a gestação (função renal, função cardíaca, fundo de olho, \ninvestigação de anemias)."" Grupo 3 exames para rastreamento de doenças potenciais na gestação, sempre \nligados às características dos serviços e do cliente, bem como as doenças prevalentes em cada região (rastreamento de diabetes, rastreamento de préeclampsia)." " Grupo 4 exames para rastreamento de doenças indiretamente ligadas à gestação. têm \nsua realização justificada pela oportunidade de estar diante da mulher que nunca teve a \noportunidade de buscar serviço médico (rastreamento de verminoses, rastreamento de \ncâncer cervical)."': 0.5995887517929077}
Exame físico ginecoobstétrico
Em termos práticos, o exame físico ginecoobstétrico nas consultas de retorno pré-natal 
deverá considerar o exame das mamas e o exame obstétrico propriamente dito.
---------------------------------------------------
input Quais condutas e orientações na gestação?
{'Condutas e orientações\nEsse também é o último tempo das consultas subsequentes à consulta inicial de pré-natal, \nno qual serão dadas à gestante todas as orientações necessárias e que facilitem sua adesão \nàs condutas e intervenções prescritas.': 0.6017743945121765, 'Durante a gestação, pelos fatores referidos, algumas manifestações clínicas são previsíveis. o \nbom senso e as condições individuais deverão nortear as orientações específicas.': 0.593505322933197, 'CONDUTAS E ORIENTAÇÕES NA PRIMEIRA CONSULTA DE pré-natal\nEsse é o último tempo da consulta de pré-natal, no qual serão transmitidas à gestante todas \nas informações que sejam necessárias e facilitem sua adesão a todas as orientações, condutas \ne intervenções prescritas.': 0.5527164936065674, 'No planejamento da gestação, a mulher deve ser orientada a abandonar esses hábitos e vícios.': 0.5805822610855103, 'Orientação sobre hábitos higiênicos e vestes\nEssas orientações parecem dispensáveis para gestantes bem informadas, mas serão de grande \nutilidade para aquelas sem essas informações.': 0.5978624820709229, 'A seguir são apresentados os diversos itens que os profissionais de saúde devem conferir na observação de uma mamada\nAs roupas da mãe e do bebê são adequadas, sem restringir movimentos?': 0.5657435655593872, 'O profissional de saúde deve avaliar se os mamilos estão invertidos ou não, o que vai ditar as orientações e informações \nque ele precisa fornecer à gestante quanto aos cuidados com as mamas e a amamentação.': 0.6128723621368408, 'Sobre o uso de medicamentos, Orienta-se a gestante a não ingerilos sem prescrição médica.': 0.5558966398239136, 'A proposta contida neste Guia Alimentar traz orientações de como \nproceder para ultrapassar estes problemas.': 0.5611878037452698, 'No entanto, a depender do tipo \nde trabalho, devem ser dadas orientações pontuais.': 0.5907381772994995}
O profissional de saúde deve avaliar se os mamilos estão invertidos ou não, o que vai ditar as orientações e informações 
que ele precisa fornecer à gestante quanto aos cuidados com as mamas e a amamentação.
---------------------------------------------------
input Ecografia
{'Ecocardiografia fetal\nA ecocardiografia fetal é um exame anatômico e funcional do coração fetal."Pode ser \nrealizado a partir de 18 semanas de gestação, mas o período ideal é o da 28a semana."': 0.4815100431442261, 'Para o presente Guia Alimentar adotouse o gráfico na forma de \nPirâmide.': 0.474643349647522, 'Para padronizar um estudo deste tipo, os Drs.': 0.434245765209198, 'O exame das mamas iniciase com a inspeção estática, seguida da inspeção dinâmica.': 0.4420710802078247, 'Dividir o exame físico na primeira consulta de pré-natal em geral, especial e ginecoobstétrico não é apenas uma alusão didática, mas deve nortear, nessa sequência, sua sistematização considerando seus aspectos práticos.': 0.43852198123931885, 'Os três tipos de métodos classificatórios mais empregados são a curva ponderal da gestante (curva de Rosso), a curva do Centro Latinoamericano de Perinatologia (CLAP) e o método de Atalah.': 0.43902814388275146, 'Estudos adicionais que incluem um maior número de \nlocais representativos das diversas regiões do País são desejáveis, se possível \ncom subamostras para observação direta da freqüência de alimentação.': 0.4406052827835083, 'Manobras de LeopoldZweifel, modificada do Manual técnico de pré-natal e puerpério (MS, 2005)\nA anotação das medidas de altura uterina em gráfico proporciona uma visão do \ndesenvolvimento fetal nas curvas dos percentis 10, 50 e 90.': 0.41892075538635254, 'Podemos dividir o exame clínico em \nsemiologia obstétrica e semiologia não obstétrica, como descreveremos a seguir.': 0.45251011848449707, 'Exame físico ginecológico deve-se seguir o padrão habitual de todo exame \nginecológico.': 0.43735092878341675}
Ecocardiografia fetal
A ecocardiografia fetal é um exame anatômico e funcional do coração fetal."Pode ser 
realizado a partir de 18 semanas de gestação, mas o período ideal é o da 28a semana."
---------------------------------------------------

In [ ]:
nltk.download('omw')
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = spacy.load('pt')
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
text = "Eu quero retirar 5.000 euros"
economy_domains = ['finance', 'banking']
enriched_sentence = []
sentence = nlp(text)

# For each token in the sentence
for token in sentence:
    # We get those synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
    if not synsets:
        enriched_sentence.append(token.text)
    else:
        lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names('por')]
        # If we found a synset in the economy domains
        # we get the variants and add them to the enriched sentence
        enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))

# Let's see our enriched sentence
print(' '.join(enriched_sentence))
# >> Eu (querer|desejar|esperar) retirar 5.000 euros

[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
Eu (querer|esperar|desejar) retirar 5.000 euros
